In [1]:
#Monomeric features
import pandas as pd
import numpy as np
import ast

In [8]:
df_train = pd.read_csv('/home/users/akshay/PCPpred/Caco2/data/Train_Caco2.csv')
df_train

,ID,SMILES,Permeability,Sequence,MolWt
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,"['meA', 'meF', 'I', 'meA', 'meL', 'meL', 'T', ...",1664.156
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,"['dA', 'T', 'meF', 'I', 'meA', 'meL', 'meF', '...",1650.129
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,"['dL', 'Me_Phe(3-Cl)', 'L', 'meL', 'T', 'Bu_Gl...",1587.863
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,"['Me_dA', 'meA', 'meF', 'T', 'meA', 'meL', 'I'...",1574.031
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,"['Bu_Gly', 'Me_Phe(3-Cl)', 'Gln(Mes)', 'meL', ...",1567.445
...,...,...,...,...,...
1003,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,"['A', 'iBu_Gly', 'F', '5-Ava']",430.549
1004,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60,"['A', 'L', 'Mono77']",416.566
1005,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70,"['A', 'L', 'Mono76']",402.539
1006,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90,"['A', 'L', 'Mono75']",388.512


In [9]:
df_train = df_train[['ID','SMILES','Sequence','Permeability']]
df_train.loc[:,'Sequence'] = df_train['Sequence'].apply(ast.literal_eval)

In [10]:
symbol_list = []

with open('/home/users/akshay/PCPpred/data/monomer_list.txt', 'r') as file:
    for line in file:
        symbol_list.append(line.strip())
print(symbol_list)

['A', 'dA', 'meA', 'Me_dA', 'Ala(tBu)', 'Ala(indol-2-yl)', 'dAla(indol-2-yl)', 'Me_Ala(indol-2-yl)', 'Ala(5-Tet)', 'Abu', 'dAbu', 'Me_Abu', 'Me_dAbu', 'Me_Abu(morpholino)', '2Abz', 'Aib', 'Aoc(2)', '5-Ava', 'Bal', 'Me_Bal', 'HOCOCH2_Bal', 'Cys(EtO2H)_NH2', 'Cha', 'dCha', 'Me_Cha', 'D', 'meD', 'Asp_piperidide', 'Asp(OMe)', 'Asp(Ph(2-NH2))', 'dAsp(pyrrol-1-yl)', 'E', 'Glu_NH2', 'Glu(3R-Me)', 'Glu(OMe)', 'dGlu(OMe)', 'F', 'dF', 'meF', 'Me_dF', 'Phe(4-F)', 'dPhe(4-F)', 'Phe(4-CF3)', 'Phe(4-NO2)', 'Phe(CHF2)', 'dPhe(3,4-diF)', 'Et_Phe', 'H2NEt_Phe', 'Me_Phe(3-Cl)', 'Me_Phe(4-Cl)', 'Me_Phe(a,b-dehydro)', 'G', 'Bn_Gly', 'Bn(4-Cl)_Gly', 'Bn(4-OH)_Gly', 'Bu_Gly', 'iBu_Gly', 'Et_Gly', 'EtOEt_Gly', 'HOCOCH2_Gly_ol', 'MeOEt_Gly', 'NH2Bu_Gly', 'Pr_Gly', 'PhEt_Gly', 'PhPr_Gly', 'cHexCH2_Gly', 'isoamyl_Gly', 'pentyl_Gly', '3-pyridylethyl_Gly', '2-pyridylmethyl_Gly', 'd(N->O)Gly(allyl)', 'GABA', 'H', 'Hph', 'Me_Hph', 'bHph', 'Hph(2-Cl)', 'Hph(3-Cl)', 'Hph(4-Cl)', 'Hse(Et)', 'dHyp', 'Hyp(Et)', 'I', 'dI

In [11]:
def calculate_monomer_composition(df, symbol_list):
    composition_data = []

    for index, row in df.iterrows():
        # print(row['Sequence'])
        composition = {symbol: 0 for symbol in symbol_list}

        for s in row['Sequence']:
            if s in composition:
                # print(s)
                composition[s] += 1

        total_length = len(row['Sequence'])
        # print(total_length)

        frequency = {symbol: count / total_length for symbol, count in composition.items()}

        composition_data.append({
            'ID': row['ID'],
            'SMILES': row['SMILES'],
            'Permeability': row['Permeability'],
            **frequency 
        })

    composition_df = pd.DataFrame(composition_data)

    return composition_df

In [12]:
df_train_mc = calculate_monomer_composition(df_train, symbol_list)
df_train_mc.shape

(1008, 388)

In [13]:
df_train_mc.to_csv("/home/users/akshay/PCPpred/Caco2/features/Monomeric/Train_mon_comp_caco2.csv", index=False)

In [36]:
df_test = pd.read_csv('/home/users/akshay/PCPpred/Caco2/data/Test_Caco2.csv')

df_test = df_test[['ID','SMILES','Sequence','Permeability']]
df_test.loc[:,'Sequence'] = df_test['Sequence'].apply(ast.literal_eval)

df_test_mc = calculate_monomer_composition(df_test, symbol_list)
df_test_mc.shape

df_test_mc.to_csv("/home/users/akshay/PCPpred/Caco2/features/Monomeric/Test_mon_comp_Caco2.csv", index=False)
df_test_mc

,ID,SMILES,Permeability,A,dA,meA,Me_dA,Ala(tBu),Ala(indol-2-yl),dAla(indol-2-yl),...,Mono118,Mono119,Mono120,Mono121,Mono122,Mono123,Mono124,Mono125,Mono126,Mono127
0,2064,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.19,0.071429,0.000000,0.071429,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,8066,CC[C@H]1C(=O)N[C@@H](COCCC(C)C)C(=O)N(C)[C@@H]...,-6.21,0.000000,0.000000,0.000000,0.083333,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,2068,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-7.24,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,2234,CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[...,-5.85,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,2230,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[...,-5.96,0.000000,0.090909,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,8448,CC(C)C[C@@H]1NC(=O)CN(C)C(=O)[C@H](Cc2ccccc2)N...,-5.88,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0
248,2478,CC(C)C[C@@H]1NC(=O)[C@H](C)NCCCCCCNC(=O)[C@H](...,-4.50,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
249,2477,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-4.20,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
250,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,0.333333,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [16]:
#Amino acid composition 20 natural and X as non natural amino acids

In [17]:
mono = pd.read_csv('/home/users/akshay/PCPpred/data/CycPeptMPDB_Monomer_All (2).csv')
mono = mono[['Symbol','Natural_Analog']]
mono_dict = mono.set_index('Symbol')['Natural_Analog'].to_dict()
mono_dict

{'A': 'A',
 'dA': 'A',
 'meA': 'A',
 'Me_dA': 'A',
 'Ala(tBu)': 'A',
 'Ala(indol-2-yl)': 'A',
 'dAla(indol-2-yl)': 'A',
 'Me_Ala(indol-2-yl)': 'A',
 'Ala(5-Tet)': 'A',
 'Abu': 'X',
 'dAbu': 'X',
 'Me_Abu': 'X',
 'Me_dAbu': 'X',
 'Me_Abu(morpholino)': 'X',
 '2Abz': 'X',
 'Aib': 'X',
 'Aoc(2)': 'L',
 '5-Ava': 'X',
 'Bal': 'A',
 'Me_Bal': 'A',
 'HOCOCH2_Bal': 'A',
 'Cys(EtO2H)_NH2': 'C',
 'Cha': 'A',
 'dCha': 'A',
 'Me_Cha': 'A',
 'D': 'D',
 'meD': 'D',
 'Asp_piperidide': 'D',
 'Asp(OMe)': 'D',
 'Asp(Ph(2-NH2))': 'D',
 'dAsp(pyrrol-1-yl)': 'D',
 'E': 'E',
 'Glu_NH2': 'E',
 'Glu(3R-Me)': 'E',
 'Glu(OMe)': 'E',
 'dGlu(OMe)': 'E',
 'F': 'F',
 'dF': 'F',
 'meF': 'F',
 'Me_dF': 'F',
 'Phe(4-F)': 'F',
 'dPhe(4-F)': 'F',
 'Phe(4-CF3)': 'F',
 'Phe(4-NO2)': 'F',
 'Phe(CHF2)': 'F',
 'dPhe(3,4-diF)': 'F',
 'Et_Phe': 'F',
 'H2NEt_Phe': 'F',
 'Me_Phe(3-Cl)': 'F',
 'Me_Phe(4-Cl)': 'F',
 'Me_Phe(a,b-dehydro)': 'F',
 'G': 'G',
 'Bn_Gly': 'G',
 'Bn(4-Cl)_Gly': 'G',
 'Bn(4-OH)_Gly': 'G',
 'Bu_Gly': 'G',
 '

In [18]:
amino_acid_list =  list(set(mono['Natural_Analog'].to_list()))
amino_acid_list

['I',
 'Q',
 'W',
 'H',
 'D',
 'A',
 'F',
 'X',
 'S',
 'C',
 'L',
 'N',
 'R',
 'V',
 'E',
 'P',
 'T',
 'K',
 'M',
 'G',
 'Y']

In [19]:
def seq2seq(sequence):
    seq = ''
    for s in sequence:
        if s in mono_dict.keys():
            seq += mono_dict[s]
    return seq

In [20]:
def calculate_aac(df, symbol_list):
    composition_data = []

    for index, row in df.iterrows():
        # print(row['Sequence'])
        composition = {symbol: 0 for symbol in symbol_list}

        for s in row['Natural_analog_sequence']:
            if s in composition:
                # print(s)
                composition[s] += 1

        total_length = len(row['Natural_analog_sequence'])
        # print(total_length)

        frequency = {symbol: count / total_length for symbol, count in composition.items()}

        composition_data.append({
            'ID': row['ID'],
            'SMILES': row['SMILES'],
            'Permeability': row['Permeability'],
            **frequency 
        })

    composition_df = pd.DataFrame(composition_data)

    return composition_df

In [21]:
df_train = pd.read_csv("/home/users/akshay/PCPpred/Caco2/data/Train_Caco2.csv")
df_train.loc[:,'Sequence'] = df_train['Sequence'].apply(ast.literal_eval)
df_train.loc[:, 'Natural_analog_sequence'] = df_train['Sequence'].apply(lambda x :seq2seq(x))
df_train = calculate_aac(df_train, amino_acid_list)
print('Shape',df_train.shape)
df_train.to_csv('/home/users/akshay/PCPpred/Caco2/features/Monomeric/Train_aac_Caco2.csv',index=False)
df_train

Shape (1008, 24)


,ID,SMILES,Permeability,I,Q,W,H,D,A,F,...,N,R,V,E,P,T,K,M,G,Y
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,0.071429,0.000000,0.0,0.0,0.071429,0.214286,0.285714,...,0.0,0.0,0.000000,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,0.071429,0.000000,0.0,0.0,0.071429,0.214286,0.285714,...,0.0,0.0,0.000000,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,0.000000,0.090909,0.0,0.0,0.090909,0.000000,0.181818,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,0.071429,0.000000,0.0,0.0,0.071429,0.285714,0.214286,...,0.0,0.0,0.071429,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,0.000000,0.090909,0.0,0.0,0.090909,0.000000,0.181818,...,0.0,0.0,0.000000,0.0,0.0,0.090909,0.0,0.0,0.181818,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,0.000000,0.000000,0.0,0.0,0.000000,0.250000,0.250000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.250000,0.0
1004,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60,0.000000,0.000000,0.0,0.0,0.000000,0.333333,0.333333,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
1005,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70,0.000000,0.000000,0.0,0.0,0.000000,0.333333,0.333333,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
1006,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90,0.000000,0.000000,0.0,0.0,0.000000,0.333333,0.333333,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0


In [22]:
df_test = pd.read_csv("/home/users/akshay/PCPpred/Caco2/data/Test_Caco2.csv")
df_test.loc[:,'Sequence'] = df_test['Sequence'].apply(ast.literal_eval)
df_test.loc[:, 'Natural_analog_sequence'] = df_test['Sequence'].apply(lambda x :seq2seq(x))
df_test = calculate_aac(df_test, amino_acid_list)
print('Shape',df_test.shape)
df_test.to_csv('/home/users/akshay/PCPpred/Caco2/features/Monomeric/Test_aac_Caco2.csv',index=False)
df_test

Shape (252, 24)


,ID,SMILES,Permeability,I,Q,W,H,D,A,F,...,N,R,V,E,P,T,K,M,G,Y
0,2064,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.19,0.071429,0.0,0.0,0.0,0.071429,0.214286,0.285714,...,0.0,0.0,0.000000,0.0,0.000000,0.071429,0.0,0.0,0.000000,0.0
1,8066,CC[C@H]1C(=O)N[C@@H](COCCC(C)C)C(=O)N(C)[C@@H]...,-6.21,0.000000,0.0,0.0,0.0,0.083333,0.083333,0.083333,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
2,2068,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-7.24,0.000000,0.0,0.0,0.0,0.090909,0.000000,0.363636,...,0.0,0.0,0.000000,0.0,0.000000,0.090909,0.0,0.0,0.000000,0.0
3,2234,CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[...,-5.85,0.000000,0.0,0.0,0.0,0.090909,0.000000,0.181818,...,0.0,0.0,0.181818,0.0,0.090909,0.090909,0.0,0.0,0.090909,0.0
4,2230,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[...,-5.96,0.000000,0.0,0.0,0.0,0.090909,0.090909,0.181818,...,0.0,0.0,0.000000,0.0,0.090909,0.090909,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,8448,CC(C)C[C@@H]1NC(=O)CN(C)C(=O)[C@H](Cc2ccccc2)N...,-5.88,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.333333,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.333333,0.0
248,2478,CC(C)C[C@@H]1NC(=O)[C@H](C)NCCCCCCNC(=O)[C@H](...,-4.50,0.000000,0.0,0.0,0.0,0.000000,0.333333,0.333333,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
249,2477,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-4.20,0.000000,0.0,0.0,0.0,0.000000,0.333333,0.333333,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
250,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,0.000000,0.0,0.0,0.0,0.000000,0.333333,0.333333,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0


In [145]:
#Atomic features
import pandas as pd
import numpy as np
from rdkit import Chem

In [146]:
def get_unique_atoms(smiles_list):
    unique_atoms = set()
    
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:  # Check if the molecule was created successfully
            for atom in mol.GetAtoms():
                unique_atoms.add(atom.GetSymbol())  # Add atomic symbol to the set
    
    return unique_atoms

In [147]:
def get_atomic_composition_frequency(smiles_list):
    atomic_frequencies = []
    
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        composition = {'Br': 0, 'C': 0, 'Cl': 0, 'F': 0, 'N': 0, 'O': 0, 'S': 0}  
        
        if mol is not None:  # Check if the molecule was created successfully
            total_atoms = mol.GetNumAtoms()  # Get total number of atoms in the molecule
            
            for atom in mol.GetAtoms():
                composition[atom.GetSymbol()] += 1  # Count occurrences of each atom
            
            # Calculate frequency
            frequency = {atom: count / total_atoms for atom, count in composition.items()}
            atomic_frequencies.append(frequency)  # Append the frequency for this SMILES
    
    return atomic_frequencies

In [148]:
#Degree of atoms
target_atoms = {'Br', 'C', 'Cl', 'F', 'N', 'O', 'S'}
def compute_target_atom_degrees(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None  # Handle invalid SMILES

    # Initialize a dictionary to hold the highest degrees for target atoms
    degrees_dict = {atom: 0 for atom in target_atoms}

    # Get the degree of each atom and update the dictionary for target atoms
    for atom in mol.GetAtoms():
        atom_symbol = atom.GetSymbol()
        if atom_symbol in target_atoms:
            current_degree = atom.GetDegree()
            # Update the degree if the current one is higher
            if current_degree > degrees_dict[atom_symbol]:
                degrees_dict[atom_symbol] = current_degree

    return degrees_dict

In [149]:
#Bond type
def compute_bond_types_for_cyclic_peptides(df):
    
    def compute_bond_types(smiles):
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None  # Handle invalid SMILES

        bond_types = {
            'Single': 0,
            'Double': 0,
            'Triple': 0,
            'Aromatic': 0,
            'Conjugated': 0,
            'No-bond': 0
        }

        # Iterate through the bonds in the molecule
        for bond in mol.GetBonds():
            bond_order = bond.GetBondTypeAsDouble()
            if bond_order == 1.0:
                bond_types['Single'] += 1
            elif bond_order == 2.0:
                bond_types['Double'] += 1
            elif bond_order == 3.0:
                bond_types['Triple'] += 1
            elif bond_order == 1.5:
                bond_types['Aromatic'] += 1
            elif bond_order == 1.4:
                bond_types['Conjugated'] += 1
            else:
                bond_types['No-bond'] += 1

        return bond_types

    df['Bond_Types'] = df['SMILES'].apply(compute_bond_types)

    bond_types_df = df['Bond_Types'].apply(pd.Series)

    df = pd.concat([df, bond_types_df], axis=1)

    df.drop(columns=['Bond_Types'], inplace=True)

    return df

In [150]:
#Formal charges
def calculate_overall_formal_charge(smiles):
   
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None  # Handle invalid SMILES

    overall_charge = 0
    
    # Define valence electrons for common atoms
    valence_electrons = {
        'C': 4,
        'N': 5,
        'O': 6,
        'S': 6,
        'P': 5,
        'F': 7,
        'Cl': 7,
        'Br': 7,
        'I': 7,
    }

    for atom in mol.GetAtoms():
        atom_symbol = atom.GetSymbol()
        valence = valence_electrons.get(atom_symbol, 0)
        non_bonding = atom.GetNumImplicitHs() 
        bonding = atom.GetDegree() * 2  # 2 electrons for each bond

        # Calculate formal charge
        formal_charge = valence - (non_bonding + bonding // 2)
        overall_charge += formal_charge

    return overall_charge

In [151]:
def check_aromatic_and_ring(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return (None, None)  # Handle invalid SMILES

    # Check if the molecule is aromatic
    is_aromatic = any(atom.GetIsAromatic() for atom in mol.GetAtoms())

    # Check if the molecule contains a ring
    ring_info = mol.GetRingInfo()
    is_in_ring = ring_info.NumRings() > 0

    return (int(is_aromatic), int(is_in_ring))

In [152]:
df_train = pd.read_csv('/home/users/akshay/PCPpred/Caco2/data/Train_Caco2.csv')
df_train

,ID,SMILES,Permeability,Sequence,MolWt
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,"['meA', 'meF', 'I', 'meA', 'meL', 'meL', 'T', ...",1664.156
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,"['dA', 'T', 'meF', 'I', 'meA', 'meL', 'meF', '...",1650.129
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,"['dL', 'Me_Phe(3-Cl)', 'L', 'meL', 'T', 'Bu_Gl...",1587.863
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,"['Me_dA', 'meA', 'meF', 'T', 'meA', 'meL', 'I'...",1574.031
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,"['Bu_Gly', 'Me_Phe(3-Cl)', 'Gln(Mes)', 'meL', ...",1567.445
...,...,...,...,...,...
1003,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,"['A', 'iBu_Gly', 'F', '5-Ava']",430.549
1004,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60,"['A', 'L', 'Mono77']",416.566
1005,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70,"['A', 'L', 'Mono76']",402.539
1006,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90,"['A', 'L', 'Mono75']",388.512


In [153]:
unique_atoms = get_unique_atoms(df_train['SMILES'])
unique_atoms

{'C', 'Cl', 'F', 'N', 'O', 'S'}

In [154]:
atomic_frequencies = get_atomic_composition_frequency(df_train['SMILES'])
frequency_df = pd.DataFrame(atomic_frequencies)
frequency_df.fillna(0, inplace=True)
frequency_df

,Br,C,Cl,F,N,O,S
0,0.0,0.758333,0.000000,0.0,0.116667,0.125000,0.000000
1,0.0,0.756303,0.000000,0.0,0.117647,0.126050,0.000000
2,0.0,0.706422,0.018349,0.0,0.119266,0.146789,0.009174
3,0.0,0.743363,0.000000,0.0,0.123894,0.132743,0.000000
4,0.0,0.715596,0.009174,0.0,0.119266,0.146789,0.009174
...,...,...,...,...,...,...,...
1003,0.0,0.741935,0.000000,0.0,0.129032,0.129032,0.000000
1004,0.0,0.766667,0.000000,0.0,0.133333,0.100000,0.000000
1005,0.0,0.758621,0.000000,0.0,0.137931,0.103448,0.000000
1006,0.0,0.750000,0.000000,0.0,0.142857,0.107143,0.000000


In [155]:
df_train_atomic_comp = pd.concat([df_train[['ID','SMILES','Permeability']], frequency_df], axis=1)
df_train_atomic_comp

,ID,SMILES,Permeability,Br,C,Cl,F,N,O,S
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,0.0,0.758333,0.000000,0.0,0.116667,0.125000,0.000000
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,0.0,0.756303,0.000000,0.0,0.117647,0.126050,0.000000
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,0.0,0.706422,0.018349,0.0,0.119266,0.146789,0.009174
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,0.0,0.743363,0.000000,0.0,0.123894,0.132743,0.000000
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,0.0,0.715596,0.009174,0.0,0.119266,0.146789,0.009174
...,...,...,...,...,...,...,...,...,...,...
1003,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,0.0,0.741935,0.000000,0.0,0.129032,0.129032,0.000000
1004,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60,0.0,0.766667,0.000000,0.0,0.133333,0.100000,0.000000
1005,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70,0.0,0.758621,0.000000,0.0,0.137931,0.103448,0.000000
1006,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90,0.0,0.750000,0.000000,0.0,0.142857,0.107143,0.000000


In [156]:
df_train_atomic_comp.to_csv('/home/users/akshay/PCPpred/Caco2/features/Atomic/Train_atomic_comp_Caco2.csv', index=False)

In [157]:
df_test = pd.read_csv('/home/users/akshay/PCPpred/Caco2/data/Test_Caco2.csv')
atomic_frequencies = get_atomic_composition_frequency(df_test['SMILES'])
frequency_df = pd.DataFrame(atomic_frequencies)
frequency_df.fillna(0, inplace=True)
frequency_df

,Br,C,Cl,F,N,O,S
0,0.0,0.758333,0.000000,0.000000,0.116667,0.125000,0.0
1,0.0,0.720721,0.000000,0.000000,0.108108,0.171171,0.0
2,0.0,0.767857,0.000000,0.000000,0.107143,0.125000,0.0
3,0.0,0.722222,0.009259,0.018519,0.120370,0.129630,0.0
4,0.0,0.733945,0.000000,0.018349,0.119266,0.128440,0.0
...,...,...,...,...,...,...,...
247,0.0,0.750000,0.000000,0.000000,0.125000,0.125000,0.0
248,0.0,0.774194,0.000000,0.000000,0.129032,0.096774,0.0
249,0.0,0.774194,0.000000,0.000000,0.129032,0.096774,0.0
250,0.0,0.774194,0.000000,0.000000,0.129032,0.096774,0.0


In [158]:
df_test_atomic_comp = pd.concat([df_test[['ID','SMILES','Permeability']], frequency_df], axis=1)
df_test_atomic_comp

,ID,SMILES,Permeability,Br,C,Cl,F,N,O,S
0,2064,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.19,0.0,0.758333,0.000000,0.000000,0.116667,0.125000,0.0
1,8066,CC[C@H]1C(=O)N[C@@H](COCCC(C)C)C(=O)N(C)[C@@H]...,-6.21,0.0,0.720721,0.000000,0.000000,0.108108,0.171171,0.0
2,2068,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-7.24,0.0,0.767857,0.000000,0.000000,0.107143,0.125000,0.0
3,2234,CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[...,-5.85,0.0,0.722222,0.009259,0.018519,0.120370,0.129630,0.0
4,2230,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[...,-5.96,0.0,0.733945,0.000000,0.018349,0.119266,0.128440,0.0
...,...,...,...,...,...,...,...,...,...,...
247,8448,CC(C)C[C@@H]1NC(=O)CN(C)C(=O)[C@H](Cc2ccccc2)N...,-5.88,0.0,0.750000,0.000000,0.000000,0.125000,0.125000,0.0
248,2478,CC(C)C[C@@H]1NC(=O)[C@H](C)NCCCCCCNC(=O)[C@H](...,-4.50,0.0,0.774194,0.000000,0.000000,0.129032,0.096774,0.0
249,2477,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-4.20,0.0,0.774194,0.000000,0.000000,0.129032,0.096774,0.0
250,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,0.0,0.774194,0.000000,0.000000,0.129032,0.096774,0.0


In [159]:
df_test_atomic_comp.to_csv('/home/users/akshay/PCPpred/Caco2/features/Atomic/Test_atomic_comp_Caco2.csv', index=False)

In [160]:
#Degree of atoms
df_train = pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Atomic/Train_atomic_comp_Caco2.csv')
df_test = pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Atomic/Test_atomic_comp_Caco2.csv')

In [161]:
df_train['Atom_Degrees'] = df_train['SMILES'].apply(compute_target_atom_degrees)
degrees_df = df_train['Atom_Degrees'].apply(pd.Series)

degrees_df.columns = [f'Degree_{atom}' for atom in target_atoms]

df_train = pd.concat([df_train, degrees_df], axis=1)
df_train = df_train.drop('Atom_Degrees', axis=1)
df_train.to_csv("/home/users/akshay/PCPpred/Caco2/features/Atomic/Train_atomic_comp_and_degree_Caco2.csv",index=False)
df_train

,ID,SMILES,Permeability,Br,C,Cl,F,N,O,S,Degree_Br,Degree_Cl,Degree_C,Degree_O,Degree_S,Degree_N,Degree_F
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,0.0,0.758333,0.000000,0.0,0.116667,0.125000,0.000000,0,0,3,1,0,3,0
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,0.0,0.756303,0.000000,0.0,0.117647,0.126050,0.000000,0,0,3,1,0,3,0
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,0.0,0.706422,0.018349,0.0,0.119266,0.146789,0.009174,0,1,3,1,4,3,0
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,0.0,0.743363,0.000000,0.0,0.123894,0.132743,0.000000,0,0,3,1,0,3,0
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,0.0,0.715596,0.009174,0.0,0.119266,0.146789,0.009174,0,1,3,1,4,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,0.0,0.741935,0.000000,0.0,0.129032,0.129032,0.000000,0,0,3,1,0,3,0
1004,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60,0.0,0.766667,0.000000,0.0,0.133333,0.100000,0.000000,0,0,3,1,0,2,0
1005,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70,0.0,0.758621,0.000000,0.0,0.137931,0.103448,0.000000,0,0,3,1,0,2,0
1006,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90,0.0,0.750000,0.000000,0.0,0.142857,0.107143,0.000000,0,0,3,1,0,2,0


In [162]:
df_test['Atom_Degrees'] = df_test['SMILES'].apply(compute_target_atom_degrees)
degrees_df = df_test['Atom_Degrees'].apply(pd.Series)

degrees_df.columns = [f'Degree_{atom}' for atom in target_atoms]

df_test = pd.concat([df_test, degrees_df], axis=1)
df_test = df_test.drop('Atom_Degrees', axis=1)
df_test.to_csv("/home/users/akshay/PCPpred/Caco2/features/Atomic/Test_atomic_comp_and_degree_Caco2.csv",index=False)
df_test

,ID,SMILES,Permeability,Br,C,Cl,F,N,O,S,Degree_Br,Degree_Cl,Degree_C,Degree_O,Degree_S,Degree_N,Degree_F
0,2064,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.19,0.0,0.758333,0.000000,0.000000,0.116667,0.125000,0.0,0,0,3,1,0,3,0
1,8066,CC[C@H]1C(=O)N[C@@H](COCCC(C)C)C(=O)N(C)[C@@H]...,-6.21,0.0,0.720721,0.000000,0.000000,0.108108,0.171171,0.0,0,0,4,2,0,3,0
2,2068,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-7.24,0.0,0.767857,0.000000,0.000000,0.107143,0.125000,0.0,0,0,4,2,0,3,0
3,2234,CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[...,-5.85,0.0,0.722222,0.009259,0.018519,0.120370,0.129630,0.0,0,1,4,2,0,3,1
4,2230,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[...,-5.96,0.0,0.733945,0.000000,0.018349,0.119266,0.128440,0.0,0,0,4,2,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,8448,CC(C)C[C@@H]1NC(=O)CN(C)C(=O)[C@H](Cc2ccccc2)N...,-5.88,0.0,0.750000,0.000000,0.000000,0.125000,0.125000,0.0,0,0,3,2,0,3,0
248,2478,CC(C)C[C@@H]1NC(=O)[C@H](C)NCCCCCCNC(=O)[C@H](...,-4.50,0.0,0.774194,0.000000,0.000000,0.129032,0.096774,0.0,0,0,3,1,0,2,0
249,2477,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-4.20,0.0,0.774194,0.000000,0.000000,0.129032,0.096774,0.0,0,0,3,1,0,2,0
250,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,0.0,0.774194,0.000000,0.000000,0.129032,0.096774,0.0,0,0,3,1,0,2,0


In [163]:
#CSV with only degree columns
df_train = pd.read_csv('/home/users/akshay/PCPpred/Caco2/data/Train_Caco2.csv')
df_train = df_train[['ID','SMILES','Permeability']]
df_train['Atom_Degrees'] = df_train['SMILES'].apply(compute_target_atom_degrees)
degrees_df = df_train['Atom_Degrees'].apply(pd.Series)

degrees_df.columns = [f'Degree_{atom}' for atom in target_atoms]

df_train = pd.concat([df_train, degrees_df], axis=1)
df_train = df_train.drop('Atom_Degrees', axis=1)
df_train.to_csv("/home/users/akshay/PCPpred/Caco2/features/Atomic/Train_atomic_degrees_Caco2.csv",index=False)
df_train

,ID,SMILES,Permeability,Degree_Br,Degree_Cl,Degree_C,Degree_O,Degree_S,Degree_N,Degree_F
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,0,0,3,1,0,3,0
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,0,0,3,1,0,3,0
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,0,1,3,1,4,3,0
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,0,0,3,1,0,3,0
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,0,1,3,1,4,3,0
...,...,...,...,...,...,...,...,...,...,...
1003,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,0,0,3,1,0,3,0
1004,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60,0,0,3,1,0,2,0
1005,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70,0,0,3,1,0,2,0
1006,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90,0,0,3,1,0,2,0


In [164]:
#CSV with only degree columns
df_test = pd.read_csv('/home/users/akshay/PCPpred/Caco2/data/Test_Caco2.csv')
df_test = df_test[['ID','SMILES','Permeability']]
df_test['Atom_Degrees'] = df_test['SMILES'].apply(compute_target_atom_degrees)
degrees_df = df_test['Atom_Degrees'].apply(pd.Series)

# Rename the columns to include the atom symbols
degrees_df.columns = [f'Degree_{atom}' for atom in target_atoms]

# Concatenate the original DataFrame with the new degrees DataFrame
df_test = pd.concat([df_test, degrees_df], axis=1)
df_test = df_test.drop('Atom_Degrees', axis=1)
df_test.to_csv("/home/users/akshay/PCPpred/Caco2/features/Atomic/Test_atomic_degrees_Caco2.csv",index=False)
df_test

,ID,SMILES,Permeability,Degree_Br,Degree_Cl,Degree_C,Degree_O,Degree_S,Degree_N,Degree_F
0,2064,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.19,0,0,3,1,0,3,0
1,8066,CC[C@H]1C(=O)N[C@@H](COCCC(C)C)C(=O)N(C)[C@@H]...,-6.21,0,0,4,2,0,3,0
2,2068,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-7.24,0,0,4,2,0,3,0
3,2234,CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[...,-5.85,0,1,4,2,0,3,1
4,2230,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[...,-5.96,0,0,4,2,0,3,1
...,...,...,...,...,...,...,...,...,...,...
247,8448,CC(C)C[C@@H]1NC(=O)CN(C)C(=O)[C@H](Cc2ccccc2)N...,-5.88,0,0,3,2,0,3,0
248,2478,CC(C)C[C@@H]1NC(=O)[C@H](C)NCCCCCCNC(=O)[C@H](...,-4.50,0,0,3,1,0,2,0
249,2477,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-4.20,0,0,3,1,0,2,0
250,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,0,0,3,1,0,2,0


In [165]:
#Bond_type
df_train = pd.read_csv('/home/users/akshay/PCPpred/Caco2/data/Train_Caco2.csv')
df_train = df_train[['ID','SMILES','Permeability']]
df_train = compute_bond_types_for_cyclic_peptides(df_train)
df_train.to_csv('/home/users/akshay/PCPpred/Caco2/features/Atomic/Train_bonds_type_Caco2.csv', index=False)


In [166]:
df_train = pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Atomic/Train_atomic_comp_and_degree_Caco2.csv')
df_train = compute_bond_types_for_cyclic_peptides(df_train)
df_train.to_csv('/home/users/akshay/PCPpred/Caco2/features/Atomic/Train_Atomic_comp_atomic_degree_bonds_type_Caco2.csv', index=False)
df_train

,ID,SMILES,Permeability,Br,C,Cl,F,N,O,S,...,Degree_O,Degree_S,Degree_N,Degree_F,Single,Double,Triple,Aromatic,Conjugated,No-bond
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,0.0,0.758333,0.000000,0.0,0.116667,0.125000,0.000000,...,1,0,3,0,87,14,0,24,0,0
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,0.0,0.756303,0.000000,0.0,0.117647,0.126050,0.000000,...,1,0,3,0,86,14,0,24,0,0
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,0.0,0.706422,0.018349,0.0,0.119266,0.146789,0.009174,...,1,4,3,0,85,15,0,12,0,0
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,0.0,0.743363,0.000000,0.0,0.123894,0.132743,0.000000,...,1,0,3,0,85,14,0,18,0,0
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,0.0,0.715596,0.009174,0.0,0.119266,0.146789,0.009174,...,1,4,3,0,85,15,0,12,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,0.0,0.741935,0.000000,0.0,0.129032,0.129032,0.000000,...,1,0,3,0,22,4,0,6,0,0
1004,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60,0.0,0.766667,0.000000,0.0,0.133333,0.100000,0.000000,...,1,0,2,0,22,3,0,6,0,0
1005,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70,0.0,0.758621,0.000000,0.0,0.137931,0.103448,0.000000,...,1,0,2,0,21,3,0,6,0,0
1006,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90,0.0,0.750000,0.000000,0.0,0.142857,0.107143,0.000000,...,1,0,2,0,20,3,0,6,0,0


In [167]:
df_test = pd.read_csv('/home/users/akshay/PCPpred/Caco2/data/Test_Caco2.csv')
df_test = df_test[['ID','SMILES','Permeability']]
df_test = compute_bond_types_for_cyclic_peptides(df_test)
df_test.to_csv('/home/users/akshay/PCPpred/Caco2/features/Atomic/Test_bonds_type_Caco2.csv', index=False)
df_test

,ID,SMILES,Permeability,Single,Double,Triple,Aromatic,Conjugated,No-bond
0,2064,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.19,87,14,0,24,0,0
1,8066,CC[C@H]1C(=O)N[C@@H](COCCC(C)C)C(=O)N(C)[C@@H]...,-6.21,95,12,0,6,0,0
2,2068,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-7.24,81,12,0,24,0,0
3,2234,CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[...,-5.85,89,12,0,12,0,0
4,2230,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[...,-5.96,90,12,0,12,0,0
...,...,...,...,...,...,...,...,...,...
247,8448,CC(C)C[C@@H]1NC(=O)CN(C)C(=O)[C@H](Cc2ccccc2)N...,-5.88,20,3,0,11,0,0
248,2478,CC(C)C[C@@H]1NC(=O)[C@H](C)NCCCCCCNC(=O)[C@H](...,-4.50,23,3,0,6,0,0
249,2477,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-4.20,23,3,0,6,0,0
250,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,23,3,0,6,0,0


In [168]:
df_test = pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Atomic/Test_atomic_comp_and_degree_Caco2.csv')
df_test = compute_bond_types_for_cyclic_peptides(df_test)
df_test.to_csv('/home/users/akshay/PCPpred/Caco2/features/Atomic/Test_Atomic_comp_atomic_degree_bonds_type_Caco2.csv', index=False)
df_test

,ID,SMILES,Permeability,Br,C,Cl,F,N,O,S,...,Degree_O,Degree_S,Degree_N,Degree_F,Single,Double,Triple,Aromatic,Conjugated,No-bond
0,2064,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.19,0.0,0.758333,0.000000,0.000000,0.116667,0.125000,0.0,...,1,0,3,0,87,14,0,24,0,0
1,8066,CC[C@H]1C(=O)N[C@@H](COCCC(C)C)C(=O)N(C)[C@@H]...,-6.21,0.0,0.720721,0.000000,0.000000,0.108108,0.171171,0.0,...,2,0,3,0,95,12,0,6,0,0
2,2068,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-7.24,0.0,0.767857,0.000000,0.000000,0.107143,0.125000,0.0,...,2,0,3,0,81,12,0,24,0,0
3,2234,CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[...,-5.85,0.0,0.722222,0.009259,0.018519,0.120370,0.129630,0.0,...,2,0,3,1,89,12,0,12,0,0
4,2230,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[...,-5.96,0.0,0.733945,0.000000,0.018349,0.119266,0.128440,0.0,...,2,0,3,1,90,12,0,12,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,8448,CC(C)C[C@@H]1NC(=O)CN(C)C(=O)[C@H](Cc2ccccc2)N...,-5.88,0.0,0.750000,0.000000,0.000000,0.125000,0.125000,0.0,...,2,0,3,0,20,3,0,11,0,0
248,2478,CC(C)C[C@@H]1NC(=O)[C@H](C)NCCCCCCNC(=O)[C@H](...,-4.50,0.0,0.774194,0.000000,0.000000,0.129032,0.096774,0.0,...,1,0,2,0,23,3,0,6,0,0
249,2477,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-4.20,0.0,0.774194,0.000000,0.000000,0.129032,0.096774,0.0,...,1,0,2,0,23,3,0,6,0,0
250,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,0.0,0.774194,0.000000,0.000000,0.129032,0.096774,0.0,...,1,0,2,0,23,3,0,6,0,0


In [169]:
#Overall formal charge
df_train = pd.read_csv('/home/users/akshay/PCPpred/Caco2/data/Train_Caco2.csv')
df_train = df_train[['ID','SMILES','Permeability']]
df_train['Overall_Formal_Charge'] = df_train['SMILES'].apply(calculate_overall_formal_charge)
df_train.to_csv('/home/users/akshay/PCPpred/Caco2/features/Atomic/Train_formal_charge_Caco2.csv', index=False)
df_train

,ID,SMILES,Permeability,Overall_Formal_Charge
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,155
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,155
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,155
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,149
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,148
...,...,...,...,...
1003,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,40
1004,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60,35
1005,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70,35
1006,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90,35


In [170]:
df_test = pd.read_csv('/home/users/akshay/PCPpred/Caco2/data/Test_Caco2.csv')
df_test = df_test[['ID','SMILES','Permeability']]
df_test['Overall_Formal_Charge'] = df_test['SMILES'].apply(calculate_overall_formal_charge)
df_test.to_csv('/home/users/akshay/PCPpred/Caco2/features/Atomic/Test_formal_charge_Caco2.csv', index=False)
df_test

,ID,SMILES,Permeability,Overall_Formal_Charge
0,2064,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.19,154
1,8066,CC[C@H]1C(=O)N[C@@H](COCCC(C)C)C(=O)N(C)[C@@H]...,-6.21,141
2,2068,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-7.24,140
3,2234,CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[...,-5.85,148
4,2230,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[...,-5.96,143
...,...,...,...,...
247,8448,CC(C)C[C@@H]1NC(=O)CN(C)C(=O)[C@H](Cc2ccccc2)N...,-5.88,42
248,2478,CC(C)C[C@@H]1NC(=O)[C@H](C)NCCCCCCNC(=O)[C@H](...,-4.50,35
249,2477,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-4.20,35
250,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,35


In [171]:
#Aromatic and ring
df_train = pd.read_csv('/home/users/akshay/PCPpred/Caco2/data/Train_Caco2.csv')
df_train = df_train[['ID','SMILES','Permeability']]
df_train[['Is_Aromatic', 'Is_In_Ring']] = df_train['SMILES'].apply(check_aromatic_and_ring).apply(pd.Series)
df_train.to_csv('/home/users/akshay/PCPpred/Caco2/features/Atomic/Train_is_ring_is_aromatic_Caco2.csv', index=False)
df_train

,ID,SMILES,Permeability,Is_Aromatic,Is_In_Ring
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,1,1
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,1,1
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,1,1
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,1,1
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,1,1
...,...,...,...,...,...
1003,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,1,1
1004,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60,1,1
1005,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70,1,1
1006,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90,1,1


In [172]:
df_test = pd.read_csv('/home/users/akshay/PCPpred/Caco2/data/Test_Caco2.csv')
df_test = df_test[['ID','SMILES','Permeability']]
df_test[['Is_Aromatic', 'Is_In_Ring']] = df_test['SMILES'].apply(check_aromatic_and_ring).apply(pd.Series)
df_test.to_csv('/home/users/akshay/PCPpred/Caco2/features/Atomic/Test_is_ring_is_aromatic_Caco2.csv', index=False)
df_test

,ID,SMILES,Permeability,Is_Aromatic,Is_In_Ring
0,2064,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.19,1,1
1,8066,CC[C@H]1C(=O)N[C@@H](COCCC(C)C)C(=O)N(C)[C@@H]...,-6.21,1,1
2,2068,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-7.24,1,1
3,2234,CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[...,-5.85,1,1
4,2230,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[...,-5.96,1,1
...,...,...,...,...,...
247,8448,CC(C)C[C@@H]1NC(=O)CN(C)C(=O)[C@H](Cc2ccccc2)N...,-5.88,1,1
248,2478,CC(C)C[C@@H]1NC(=O)[C@H](C)NCCCCCCNC(=O)[C@H](...,-4.50,1,1
249,2477,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-4.20,1,1
250,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,1,1


In [173]:
df1 = pd.read_csv("/home/users/akshay/PCPpred/Caco2/features/Atomic/Train_Atomic_comp_atomic_degree_bonds_type_Caco2.csv")
df2 = pd.read_csv("/home/users/akshay/PCPpred/Caco2/features/Atomic/Train_formal_charge_Caco2.csv")
df3 = pd.read_csv("/home/users/akshay/PCPpred/Caco2/features/Atomic/Train_is_ring_is_aromatic_Caco2.csv")

merged_df = pd.merge(df1, df2, on=['ID', 'SMILES', 'Permeability'], how='inner')

# Merge the result with df3
df_train = pd.merge(merged_df, df3, on=['ID', 'SMILES', 'Permeability'], how='inner')
df_train.to_csv('/home/users/akshay/PCPpred/Caco2/features/Atomic/Train_all_atomic_desc_Caco2.csv', index=False)

In [174]:
df1 = pd.read_csv("/home/users/akshay/PCPpred/Caco2/features/Atomic/Test_Atomic_comp_atomic_degree_bonds_type_Caco2.csv")
df2 = pd.read_csv("/home/users/akshay/PCPpred/Caco2/features/Atomic/Test_formal_charge_Caco2.csv")
df3 = pd.read_csv("/home/users/akshay/PCPpred/Caco2/features/Atomic/Test_is_ring_is_aromatic_Caco2.csv")

merged_df = pd.merge(df1, df2, on=['ID', 'SMILES', 'Permeability'], how='inner')

# Merge the result with df3
df_test = pd.merge(merged_df, df3, on=['ID', 'SMILES', 'Permeability'], how='inner')
df_test.to_csv('/home/users/akshay/PCPpred/Caco2/features/Atomic/Test_all_atomic_desc_Caco2.csv', index=False)
df_test

,ID,SMILES,Permeability,Br,C,Cl,F,N,O,S,...,Degree_F,Single,Double,Triple,Aromatic,Conjugated,No-bond,Overall_Formal_Charge,Is_Aromatic,Is_In_Ring
0,2064,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.19,0.0,0.758333,0.000000,0.000000,0.116667,0.125000,0.0,...,0,87,14,0,24,0,0,154,1,1
1,8066,CC[C@H]1C(=O)N[C@@H](COCCC(C)C)C(=O)N(C)[C@@H]...,-6.21,0.0,0.720721,0.000000,0.000000,0.108108,0.171171,0.0,...,0,95,12,0,6,0,0,141,1,1
2,2068,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-7.24,0.0,0.767857,0.000000,0.000000,0.107143,0.125000,0.0,...,0,81,12,0,24,0,0,140,1,1
3,2234,CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[...,-5.85,0.0,0.722222,0.009259,0.018519,0.120370,0.129630,0.0,...,1,89,12,0,12,0,0,148,1,1
4,2230,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[...,-5.96,0.0,0.733945,0.000000,0.018349,0.119266,0.128440,0.0,...,1,90,12,0,12,0,0,143,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,8448,CC(C)C[C@@H]1NC(=O)CN(C)C(=O)[C@H](Cc2ccccc2)N...,-5.88,0.0,0.750000,0.000000,0.000000,0.125000,0.125000,0.0,...,0,20,3,0,11,0,0,42,1,1
248,2478,CC(C)C[C@@H]1NC(=O)[C@H](C)NCCCCCCNC(=O)[C@H](...,-4.50,0.0,0.774194,0.000000,0.000000,0.129032,0.096774,0.0,...,0,23,3,0,6,0,0,35,1,1
249,2477,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-4.20,0.0,0.774194,0.000000,0.000000,0.129032,0.096774,0.0,...,0,23,3,0,6,0,0,35,1,1
250,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,0.0,0.774194,0.000000,0.000000,0.129032,0.096774,0.0,...,0,23,3,0,6,0,0,35,1,1


In [ ]:
#Fingerprints

In [2]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, rdFingerprintGenerator, rdMolDescriptors
from padelpy import padeldescriptor

In [57]:
df_train = pd.read_csv("/home/users/akshay/PCPpred/Caco2/data/Train_Caco2.csv")
df_train

,ID,SMILES,Permeability,Sequence,MolWt
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,"['meA', 'meF', 'I', 'meA', 'meL', 'meL', 'T', ...",1664.156
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,"['dA', 'T', 'meF', 'I', 'meA', 'meL', 'meF', '...",1650.129
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,"['dL', 'Me_Phe(3-Cl)', 'L', 'meL', 'T', 'Bu_Gl...",1587.863
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,"['Me_dA', 'meA', 'meF', 'T', 'meA', 'meL', 'I'...",1574.031
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,"['Bu_Gly', 'Me_Phe(3-Cl)', 'Gln(Mes)', 'meL', ...",1567.445
...,...,...,...,...,...
1003,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,"['A', 'iBu_Gly', 'F', '5-Ava']",430.549
1004,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60,"['A', 'L', 'Mono77']",416.566
1005,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70,"['A', 'L', 'Mono76']",402.539
1006,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90,"['A', 'L', 'Mono75']",388.512


In [58]:
df_train_cmfp = df_train[['ID','SMILES','Permeability']]
df_train_cmfp

,ID,SMILES,Permeability
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05
...,...,...,...
1003,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38
1004,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60
1005,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70
1006,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90


In [59]:
def generate_count_morgan_fp(smiles, radius=2, nBits=2048):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        # Count Morgan fingerprint generator
        mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=radius, fpSize=nBits)
        count_fp = mfpgen.GetCountFingerprint(mol)
        
        # Convert the count fingerprint to a dense array
        dense_fp = np.zeros((nBits,), dtype=int)
        for bit, count in count_fp.GetNonzeroElements().items():
            dense_fp[bit] = count
            
        return dense_fp
    else:
        return None

In [60]:
df_train_cmfp.loc[:,'count_morgan_fp'] = df_train_cmfp['SMILES'].apply(generate_count_morgan_fp)
df_train_cmfp['count_morgan_fp']

/tmp/ipykernel_3804160/1551993060.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_cmfp.loc[:,'count_morgan_fp'] = df_train_cmfp['SMILES'].apply(generate_count_morgan_fp)


0       [0, 8, 1, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1       [0, 8, 1, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2       [0, 6, 1, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3       [0, 8, 1, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4       [0, 5, 1, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                              ...                        
1003    [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1004    [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1005    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1006    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1007    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: count_morgan_fp, Length: 1008, dtype: object

In [61]:
df = pd.DataFrame(df_train_cmfp['count_morgan_fp'].tolist(), index=df_train_cmfp.index)

df_train_cmfp = pd.concat([df_train_cmfp, df.add_prefix('count_fp_')], axis=1)
df_train_cmfp.shape

(1008, 2052)

In [62]:
df_train_cmfp.drop('count_morgan_fp', axis=1, inplace=True)
df_train_cmfp.shape

(1008, 2051)

In [63]:
df_train_cmfp.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Train/count_morgan_fp_train_Caco2.csv', index=False)

In [64]:
df_test = pd.read_csv("/home/users/akshay/PCPpred/Caco2/data/Test_Caco2.csv")
df_test_cmfp = df_test[['ID','SMILES','Permeability']]
df_test_cmfp.loc[:,'count_morgan_fp'] = df_test_cmfp['SMILES'].apply(generate_count_morgan_fp)
df_test_cmfp['count_morgan_fp']
df = pd.DataFrame(df_test_cmfp['count_morgan_fp'].tolist(), index=df_test_cmfp.index)

df_test_cmfp = pd.concat([df_test_cmfp, df.add_prefix('count_fp_')], axis=1)
df_test_cmfp.drop('count_morgan_fp', axis=1, inplace=True)
df_test_cmfp.shape

/tmp/ipykernel_3804160/1072159945.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_cmfp.loc[:,'count_morgan_fp'] = df_test_cmfp['SMILES'].apply(generate_count_morgan_fp)


(252, 2051)

In [65]:
df_test_cmfp.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Test/count_morgan_fp_test_Caco2.csv', index=False)

In [66]:
def generate_fingerprints(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        # Create a Morgan fingerprint generator
        mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2048)
        
        # Generate the Morgan fingerprint as a bit vector
        morgan_fp = mfpgen.GetFingerprint(mol)
        
        # Convert the bit vector to a dense array
        dense_fp = np.zeros((2048,), dtype=int)
        for bit in range(2048):
            dense_fp[bit] = morgan_fp[bit]
        
        return dense_fp
    else:
        print("Invalid SMILES string.")
        return None

In [67]:
df_train = pd.read_csv("/home/users/akshay/PCPpred/Caco2/data/Train_Caco2.csv")
df_mfp_train = df_train[['ID','SMILES','Permeability']]
df_mfp_train.loc[:,'Morganfingerprints'] = df_mfp_train['SMILES'].apply(generate_fingerprints)
df = pd.DataFrame(df_mfp_train['Morganfingerprints'].tolist(), index=df_mfp_train.index)

df_mfp_train = pd.concat([df_mfp_train, df.add_prefix('Morgan_fp_')], axis=1)
df_mfp_train.drop('Morganfingerprints', axis=1, inplace=True)
df_mfp_train.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Train/morgan_fp_train_Caco2.csv', index=False)
print(df_mfp_train.shape)
df_mfp_train

/tmp/ipykernel_3804160/3544633016.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mfp_train.loc[:,'Morganfingerprints'] = df_mfp_train['SMILES'].apply(generate_fingerprints)


(1008, 2051)


,ID,SMILES,Permeability,Morgan_fp_0,Morgan_fp_1,Morgan_fp_2,Morgan_fp_3,Morgan_fp_4,Morgan_fp_5,Morgan_fp_6,...,Morgan_fp_2038,Morgan_fp_2039,Morgan_fp_2040,Morgan_fp_2041,Morgan_fp_2042,Morgan_fp_2043,Morgan_fp_2044,Morgan_fp_2045,Morgan_fp_2046,Morgan_fp_2047
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1004,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1005,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1006,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
df_test = pd.read_csv("/home/users/akshay/PCPpred/Caco2/data/Test_Caco2.csv")
df_mfp_test = df_test[['ID','SMILES','Permeability']]
df_mfp_test.loc[:,'Morganfingerprints'] = df_mfp_test['SMILES'].apply(generate_fingerprints)
df = pd.DataFrame(df_mfp_test['Morganfingerprints'].tolist(), index=df_mfp_test.index)

df_mfp_test = pd.concat([df_mfp_test, df.add_prefix('Morgan_fp_')], axis=1)
df_mfp_test.drop('Morganfingerprints', axis=1, inplace=True)
df_mfp_test.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Test/morgan_fp_test_Caco2.csv', index=False)
print(df_mfp_test.shape)
df_mfp_test

/tmp/ipykernel_3804160/261239065.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mfp_test.loc[:,'Morganfingerprints'] = df_mfp_test['SMILES'].apply(generate_fingerprints)


(252, 2051)


,ID,SMILES,Permeability,Morgan_fp_0,Morgan_fp_1,Morgan_fp_2,Morgan_fp_3,Morgan_fp_4,Morgan_fp_5,Morgan_fp_6,...,Morgan_fp_2038,Morgan_fp_2039,Morgan_fp_2040,Morgan_fp_2041,Morgan_fp_2042,Morgan_fp_2043,Morgan_fp_2044,Morgan_fp_2045,Morgan_fp_2046,Morgan_fp_2047
0,2064,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.19,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,8066,CC[C@H]1C(=O)N[C@@H](COCCC(C)C)C(=O)N(C)[C@@H]...,-6.21,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,2068,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-7.24,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,2234,CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[...,-5.85,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,2230,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[...,-5.96,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,8448,CC(C)C[C@@H]1NC(=O)CN(C)C(=O)[C@H](Cc2ccccc2)N...,-5.88,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
248,2478,CC(C)C[C@@H]1NC(=O)[C@H](C)NCCCCCCNC(=O)[C@H](...,-4.50,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
249,2477,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-4.20,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
250,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
#Padel fingerprints
import glob
import os
directory = '/home/users/akshay/PCPpred/fingerprints_xml'
pattern = '*.xml'  

xml_files = glob.glob(os.path.join(directory, pattern))
xml_files.sort()
xml_files

['/home/users/akshay/PCPpred/fingerprints_xml/AtomPairs2DFingerprintCount.xml',
 '/home/users/akshay/PCPpred/fingerprints_xml/AtomPairs2DFingerprinter.xml',
 '/home/users/akshay/PCPpred/fingerprints_xml/EStateFingerprinter.xml',
 '/home/users/akshay/PCPpred/fingerprints_xml/ExtendedFingerprinter.xml',
 '/home/users/akshay/PCPpred/fingerprints_xml/Fingerprinter.xml',
 '/home/users/akshay/PCPpred/fingerprints_xml/GraphOnlyFingerprinter.xml',
 '/home/users/akshay/PCPpred/fingerprints_xml/KlekotaRothFingerprintCount.xml',
 '/home/users/akshay/PCPpred/fingerprints_xml/KlekotaRothFingerprinter.xml',
 '/home/users/akshay/PCPpred/fingerprints_xml/MACCSFingerprinter.xml',
 '/home/users/akshay/PCPpred/fingerprints_xml/PubchemFingerprinter.xml',
 '/home/users/akshay/PCPpred/fingerprints_xml/SubstructureFingerprintCount.xml',
 '/home/users/akshay/PCPpred/fingerprints_xml/SubstructureFingerprinter.xml']

In [70]:
FP_list = ['AtomPairs2DCount',
 'AtomPairs2D',
 'EState',
 'Extended',
 'Fingerprinter',
 'Graphonly',
 'KlekotaRothCount',
 'KlekotaRoth',
 'MACCS',
 'PubChem',
 'SubstructureCount',
 'Substructure']

In [71]:
fp = dict(zip(FP_list, xml_files))
fp

{'AtomPairs2DCount': '/home/users/akshay/PCPpred/fingerprints_xml/AtomPairs2DFingerprintCount.xml',
 'AtomPairs2D': '/home/users/akshay/PCPpred/fingerprints_xml/AtomPairs2DFingerprinter.xml',
 'EState': '/home/users/akshay/PCPpred/fingerprints_xml/EStateFingerprinter.xml',
 'Extended': '/home/users/akshay/PCPpred/fingerprints_xml/ExtendedFingerprinter.xml',
 'Fingerprinter': '/home/users/akshay/PCPpred/fingerprints_xml/Fingerprinter.xml',
 'Graphonly': '/home/users/akshay/PCPpred/fingerprints_xml/GraphOnlyFingerprinter.xml',
 'KlekotaRothCount': '/home/users/akshay/PCPpred/fingerprints_xml/KlekotaRothFingerprintCount.xml',
 'KlekotaRoth': '/home/users/akshay/PCPpred/fingerprints_xml/KlekotaRothFingerprinter.xml',
 'MACCS': '/home/users/akshay/PCPpred/fingerprints_xml/MACCSFingerprinter.xml',
 'PubChem': '/home/users/akshay/PCPpred/fingerprints_xml/PubchemFingerprinter.xml',
 'SubstructureCount': '/home/users/akshay/PCPpred/fingerprints_xml/SubstructureFingerprintCount.xml',
 'Substruct

In [ ]:
# df_train = pd.read_csv("/home/users/akshay/PCPpred/Caco2/data/Train_Caco2.csv")
# df_smi = df_train[['SMILES','ID']]
# df_smi.to_csv('/home/users/akshay/PCPpred/Caco2/data/train.smi', sep='\t', index=False, header=False)
# df_smi

In [75]:
#Substructure
fingerprint = 'Substructure'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Caco2/features/Fingerprints/',fingerprint,'_train.csv']) #Substructure.csv
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/Caco2/data/Train_caco2.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
descriptors

,Name,SubFP1,SubFP2,SubFP3,SubFP4,SubFP5,SubFP6,SubFP7,SubFP8,SubFP9,...,SubFP298,SubFP299,SubFP300,SubFP301,SubFP302,SubFP303,SubFP304,SubFP305,SubFP306,SubFP307
0,2065,1,1,1,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
1,2067,1,1,1,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
2,1914,1,1,1,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
3,2026,1,1,1,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
4,1920,1,1,1,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,5604,1,1,1,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
1004,2470,1,1,1,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
1005,2469,1,1,1,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
1006,2468,1,1,1,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1


In [76]:
df_Substructure_train = pd.concat([df_train[['ID','SMILES','Permeability']], descriptors], axis=1)
df_Substructure_train.head()

,ID,SMILES,Permeability,Name,SubFP1,SubFP2,SubFP3,SubFP4,SubFP5,SubFP6,...,SubFP298,SubFP299,SubFP300,SubFP301,SubFP302,SubFP303,SubFP304,SubFP305,SubFP306,SubFP307
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,2065,1,1,1,0,0,0,...,0,0,1,1,1,0,0,0,0,1
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,2067,1,1,1,0,0,0,...,0,0,1,1,1,0,0,0,0,1
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,1914,1,1,1,0,0,0,...,0,0,1,1,1,0,0,0,0,1
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,2026,1,1,1,0,0,0,...,0,0,1,1,1,0,0,0,0,1
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,1920,1,1,1,0,0,0,...,0,0,1,1,1,0,0,0,0,1


In [77]:
df_Substructure_train.drop(['Name'],axis=1,inplace=True)
df_Substructure_train.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Train/Substructure_train_Caco2.csv', index=False)
df_Substructure_train

,ID,SMILES,Permeability,SubFP1,SubFP2,SubFP3,SubFP4,SubFP5,SubFP6,SubFP7,...,SubFP298,SubFP299,SubFP300,SubFP301,SubFP302,SubFP303,SubFP304,SubFP305,SubFP306,SubFP307
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
1004,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
1005,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
1006,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1


In [78]:
# df_test = pd.read_csv("/home/users/akshay/PCPpred/Caco2/data/Test_Caco2.csv")
# df_smi = df_test[['SMILES','ID']]
# df_smi.to_csv('/home/users/akshay/PCPpred/Caco2/data/Test_caco2.smi', sep='\t', index=False, header=False)
fingerprint = 'Substructure'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Caco2/features/Fingerprints/',fingerprint,'_test.csv']) #Substructure.csv
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/Caco2/data/Test_caco2.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
df_Substructure_test = pd.concat([df_test[['ID','SMILES','Permeability']], descriptors], axis=1)
# df_Substructure_train.head()
df_Substructure_test.drop(['Name'],axis=1,inplace=True)
df_Substructure_test.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Test/Substructure_test_Caco2.csv', index=False)
df_Substructure_test

,ID,SMILES,Permeability,SubFP1,SubFP2,SubFP3,SubFP4,SubFP5,SubFP6,SubFP7,...,SubFP298,SubFP299,SubFP300,SubFP301,SubFP302,SubFP303,SubFP304,SubFP305,SubFP306,SubFP307
0,2064,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.19,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
1,8066,CC[C@H]1C(=O)N[C@@H](COCCC(C)C)C(=O)N(C)[C@@H]...,-6.21,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
2,2068,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-7.24,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
3,2234,CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[...,-5.85,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
4,2230,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[...,-5.96,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,8448,CC(C)C[C@@H]1NC(=O)CN(C)C(=O)[C@H](Cc2ccccc2)N...,-5.88,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
248,2478,CC(C)C[C@@H]1NC(=O)[C@H](C)NCCCCCCNC(=O)[C@H](...,-4.50,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
249,2477,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-4.20,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
250,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1


In [79]:
#CountSubstructure
fingerprint = 'SubstructureCount'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Caco2/features/Fingerprints/',fingerprint,'_train.csv']) #Substructure.csv
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/Caco2/data/Train_caco2.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
df_SubstructureCount_train = pd.concat([df_train[['ID','SMILES','Permeability']], descriptors], axis=1)
df_SubstructureCount_train.head()

,ID,SMILES,Permeability,Name,SubFPC1,SubFPC2,SubFPC3,SubFPC4,SubFPC5,SubFPC6,...,SubFPC298,SubFPC299,SubFPC300,SubFPC301,SubFPC302,SubFPC303,SubFPC304,SubFPC305,SubFPC306,SubFPC307
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,2065,12.0,12.0,4.0,0.0,0.0,0.0,...,0.0,0.0,77.0,77.0,28.0,0.0,0.0,0.0,0.0,70.0
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,2067,12.0,12.0,4.0,0.0,0.0,0.0,...,0.0,0.0,77.0,77.0,28.0,0.0,0.0,0.0,0.0,70.0
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,1914,12.0,14.0,5.0,0.0,0.0,0.0,...,0.0,0.0,76.0,76.0,24.0,0.0,0.0,0.0,0.0,55.0
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,2026,13.0,10.0,4.0,0.0,0.0,0.0,...,0.0,0.0,75.0,75.0,25.0,0.0,0.0,0.0,0.0,64.0
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,1920,11.0,16.0,4.0,0.0,0.0,0.0,...,0.0,0.0,76.0,76.0,26.0,0.0,0.0,0.0,0.0,53.0


In [80]:
df_SubstructureCount_train.drop(['Name'],axis=1,inplace=True)
df_SubstructureCount_train.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Train/SubstructureCount_train_Caco2.csv', index=False)
df_SubstructureCount_train

,ID,SMILES,Permeability,SubFPC1,SubFPC2,SubFPC3,SubFPC4,SubFPC5,SubFPC6,SubFPC7,...,SubFPC298,SubFPC299,SubFPC300,SubFPC301,SubFPC302,SubFPC303,SubFPC304,SubFPC305,SubFPC306,SubFPC307
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,12.0,12.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,77.0,77.0,28.0,0.0,0.0,0.0,0.0,70.0
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,12.0,12.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,77.0,77.0,28.0,0.0,0.0,0.0,0.0,70.0
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,12.0,14.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,76.0,76.0,24.0,0.0,0.0,0.0,0.0,55.0
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,13.0,10.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,75.0,75.0,25.0,0.0,0.0,0.0,0.0,64.0
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,11.0,16.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,76.0,76.0,26.0,0.0,0.0,0.0,0.0,53.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,3.0,4.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,21.0,21.0,4.0,0.0,0.0,0.0,0.0,17.0
1004,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60,3.0,5.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,22.0,22.0,4.0,0.0,0.0,0.0,0.0,17.0
1005,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70,3.0,4.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,21.0,21.0,4.0,0.0,0.0,0.0,0.0,17.0
1006,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90,3.0,3.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,20.0,20.0,4.0,0.0,0.0,0.0,0.0,17.0


In [81]:
fingerprint = 'SubstructureCount'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Caco2/features/Fingerprints/',fingerprint,'_test.csv']) #Substructure.csv
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/Caco2/data/Test_caco2.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
df_SubstructureCount_test = pd.concat([df_test[['ID','SMILES','Permeability']], descriptors], axis=1)
df_SubstructureCount_test.drop(['Name'],axis=1,inplace=True)
df_SubstructureCount_test.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Test/SubstructureCount_test_Caco2.csv', index=False)
df_SubstructureCount_test

,ID,SMILES,Permeability,SubFPC1,SubFPC2,SubFPC3,SubFPC4,SubFPC5,SubFPC6,SubFPC7,...,SubFPC298,SubFPC299,SubFPC300,SubFPC301,SubFPC302,SubFPC303,SubFPC304,SubFPC305,SubFPC306,SubFPC307
0,2064,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.19,11.0,13.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,77.0,77.0,28.0,0.0,0.0,0.0,0.0,69.0
1,8066,CC[C@H]1C(=O)N[C@@H](COCCC(C)C)C(=O)N(C)[C@@H]...,-6.21,16.0,11.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,89.0,89.0,32.0,0.0,0.0,0.0,0.0,48.0
2,2068,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-7.24,12.0,12.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,73.0,73.0,22.0,0.0,0.0,0.0,0.0,65.0
3,2234,CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[...,-5.85,12.0,11.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,77.0,77.0,17.0,0.0,0.0,0.0,0.0,55.0
4,2230,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[...,-5.96,13.0,13.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,80.0,80.0,19.0,0.0,0.0,0.0,0.0,56.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,8448,CC(C)C[C@@H]1NC(=O)CN(C)C(=O)[C@H](Cc2ccccc2)N...,-5.88,2.0,2.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,19.0,19.0,4.0,0.0,0.0,0.0,0.0,20.0
248,2478,CC(C)C[C@@H]1NC(=O)[C@H](C)NCCCCCCNC(=O)[C@H](...,-4.50,3.0,6.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,23.0,23.0,4.0,0.0,0.0,0.0,0.0,17.0
249,2477,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-4.20,3.0,6.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,23.0,23.0,4.0,0.0,0.0,0.0,0.0,17.0
250,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,3.0,6.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,23.0,23.0,4.0,0.0,0.0,0.0,0.0,17.0


In [82]:
#MACCS
fingerprint = 'MACCS'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Caco2/features/Fingerprints/',fingerprint,'_train.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/Caco2/data/Train_caco2.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
df_MACCS_train = pd.concat([df_train[['ID','SMILES','Permeability']], descriptors], axis=1)
df_MACCS_train.head()
df_MACCS_train.drop(['Name'],axis=1,inplace=True)
df_MACCS_train.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Train/MACCS_train_Caco2.csv', index=False)
df_MACCS_train

,ID,SMILES,Permeability,MACCSFP1,MACCSFP2,MACCSFP3,MACCSFP4,MACCSFP5,MACCSFP6,MACCSFP7,...,MACCSFP157,MACCSFP158,MACCSFP159,MACCSFP160,MACCSFP161,MACCSFP162,MACCSFP163,MACCSFP164,MACCSFP165,MACCSFP166
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
1004,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
1005,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
1006,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0


In [83]:
fingerprint = 'MACCS'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Caco2/features/Fingerprints/',fingerprint,'_test.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/Caco2/data/Test_caco2.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
df_MACCS_test = pd.concat([df_test[['ID','SMILES','Permeability']], descriptors], axis=1)
df_MACCS_test.drop(['Name'],axis=1,inplace=True)
df_MACCS_test.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Test/MACCS_test_Caco2.csv', index=False)
df_MACCS_test

,ID,SMILES,Permeability,MACCSFP1,MACCSFP2,MACCSFP3,MACCSFP4,MACCSFP5,MACCSFP6,MACCSFP7,...,MACCSFP157,MACCSFP158,MACCSFP159,MACCSFP160,MACCSFP161,MACCSFP162,MACCSFP163,MACCSFP164,MACCSFP165,MACCSFP166
0,2064,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.19,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
1,8066,CC[C@H]1C(=O)N[C@@H](COCCC(C)C)C(=O)N(C)[C@@H]...,-6.21,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
2,2068,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-7.24,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
3,2234,CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[...,-5.85,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
4,2230,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[...,-5.96,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,8448,CC(C)C[C@@H]1NC(=O)CN(C)C(=O)[C@H](Cc2ccccc2)N...,-5.88,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
248,2478,CC(C)C[C@@H]1NC(=O)[C@H](C)NCCCCCCNC(=O)[C@H](...,-4.50,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
249,2477,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-4.20,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
250,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0


In [84]:
#PubChem
fingerprint = 'PubChem'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Caco2/features/Fingerprints/',fingerprint,'_train.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/Caco2/data/Train_caco2.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
df_PubChem_train = pd.concat([df_train[['ID','SMILES','Permeability']], descriptors], axis=1)
df_PubChem_train.drop(['Name'],axis=1,inplace=True)
df_PubChem_train.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Train/PubChem_train_Caco2.csv', index=False)
df_PubChem_train

,ID,SMILES,Permeability,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1004,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1005,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1006,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
fingerprint = 'PubChem'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Caco2/features/Fingerprints/',fingerprint,'_test.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/Caco2/data/Test_caco2.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
df_PubChem_test = pd.concat([df_test[['ID','SMILES','Permeability']], descriptors], axis=1)
df_PubChem_test.drop(['Name'],axis=1,inplace=True)
df_PubChem_test.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Test/PubChem_test_Caco2.csv', index=False)
df_PubChem_test

,ID,SMILES,Permeability,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,2064,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.19,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8066,CC[C@H]1C(=O)N[C@@H](COCCC(C)C)C(=O)N(C)[C@@H]...,-6.21,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2068,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-7.24,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2234,CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[...,-5.85,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2230,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[...,-5.96,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,8448,CC(C)C[C@@H]1NC(=O)CN(C)C(=O)[C@H](Cc2ccccc2)N...,-5.88,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
248,2478,CC(C)C[C@@H]1NC(=O)[C@H](C)NCCCCCCNC(=O)[C@H](...,-4.50,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
249,2477,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-4.20,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
250,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
#AtomPairs2DCount
fingerprint = 'AtomPairs2DCount'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Caco2/features/Fingerprints/',fingerprint,'_train.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/Caco2/data/Train_caco2.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
df_AtomPairs2DCount_train = pd.concat([df_train[['ID','SMILES','Permeability']], descriptors], axis=1)
df_AtomPairs2DCount_train.drop(['Name'],axis=1,inplace=True)
df_AtomPairs2DCount_train.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Train/AtomPairs2DCount_train_Caco2.csv', index=False)
df_AtomPairs2DCount_train

,ID,SMILES,Permeability,APC2D1_C_C,APC2D1_C_N,APC2D1_C_O,APC2D1_C_S,APC2D1_C_P,APC2D1_C_F,APC2D1_C_Cl,...,APC2D10_I_I,APC2D10_I_B,APC2D10_I_Si,APC2D10_I_X,APC2D10_B_B,APC2D10_B_Si,APC2D10_B_X,APC2D10_Si_Si,APC2D10_Si_X,APC2D10_X_X
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,72.0,38.0,15.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,72.0,37.0,15.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,60.0,32.0,14.0,1.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,64.0,38.0,15.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,60.0,33.0,14.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,19.0,9.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1004,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60,20.0,8.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1005,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70,19.0,8.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1006,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90,18.0,8.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [87]:
fingerprint = 'AtomPairs2DCount'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Caco2/features/Fingerprints/',fingerprint,'_test.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/Caco2/data/Test_caco2.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
df_AtomPairs2DCount_test = pd.concat([df_test[['ID','SMILES','Permeability']], descriptors], axis=1)
df_AtomPairs2DCount_test.drop(['Name'],axis=1,inplace=True)
df_AtomPairs2DCount_test.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Test/AtomPairs2DCount_test_Caco2.csv', index=False)
df_AtomPairs2DCount_test

,ID,SMILES,Permeability,APC2D1_C_C,APC2D1_C_N,APC2D1_C_O,APC2D1_C_S,APC2D1_C_P,APC2D1_C_F,APC2D1_C_Cl,...,APC2D10_I_I,APC2D10_I_B,APC2D10_I_Si,APC2D10_I_X,APC2D10_B_B,APC2D10_B_Si,APC2D10_B_X,APC2D10_Si_Si,APC2D10_Si_X,APC2D10_X_X
0,2064,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.19,72.0,38.0,15.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8066,CC[C@H]1C(=O)N[C@@H](COCCC(C)C)C(=O)N(C)[C@@H]...,-6.21,59.0,30.0,24.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2068,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-7.24,71.0,31.0,15.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2234,CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[...,-5.85,60.0,35.0,15.0,0.0,0.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2230,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[...,-5.96,63.0,34.0,15.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,8448,CC(C)C[C@@H]1NC(=O)CN(C)C(=O)[C@H](Cc2ccccc2)N...,-5.88,20.0,9.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
248,2478,CC(C)C[C@@H]1NC(=O)[C@H](C)NCCCCCCNC(=O)[C@H](...,-4.50,21.0,8.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
249,2477,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-4.20,21.0,8.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
250,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,21.0,8.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [88]:
#AtomPairs2D
fingerprint = 'AtomPairs2D'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Caco2/features/Fingerprints/',fingerprint,'_train.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/Caco2/data/Train_caco2.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
df_AtomPairs2D_train = pd.concat([df_train[['ID','SMILES','Permeability']], descriptors], axis=1)
df_AtomPairs2D_train.drop(['Name'],axis=1,inplace=True)
df_AtomPairs2D_train.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Train/AtomPairs2D_train_Caco2.csv', index=False)
df_AtomPairs2D_train

,ID,SMILES,Permeability,AD2D1,AD2D2,AD2D3,AD2D4,AD2D5,AD2D6,AD2D7,...,AD2D771,AD2D772,AD2D773,AD2D774,AD2D775,AD2D776,AD2D777,AD2D778,AD2D779,AD2D780
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,1,1,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,1,1,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1004,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1005,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1006,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [89]:
fingerprint = 'AtomPairs2D'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Caco2/features/Fingerprints/',fingerprint,'_test.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/Caco2/data/Test_caco2.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
df_AtomPairs2D_test = pd.concat([df_test[['ID','SMILES','Permeability']], descriptors], axis=1)
df_AtomPairs2D_test.drop(['Name'],axis=1,inplace=True)
df_AtomPairs2D_test.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Test/AtomPairs2D_test_Caco2.csv', index=False)
df_AtomPairs2D_test

,ID,SMILES,Permeability,AD2D1,AD2D2,AD2D3,AD2D4,AD2D5,AD2D6,AD2D7,...,AD2D771,AD2D772,AD2D773,AD2D774,AD2D775,AD2D776,AD2D777,AD2D778,AD2D779,AD2D780
0,2064,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.19,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8066,CC[C@H]1C(=O)N[C@@H](COCCC(C)C)C(=O)N(C)[C@@H]...,-6.21,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2068,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-7.24,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2234,CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[...,-5.85,1,1,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,2230,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[...,-5.96,1,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,8448,CC(C)C[C@@H]1NC(=O)CN(C)C(=O)[C@H](Cc2ccccc2)N...,-5.88,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
248,2478,CC(C)C[C@@H]1NC(=O)[C@H](C)NCCCCCCNC(=O)[C@H](...,-4.50,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
249,2477,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-4.20,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
250,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
#EState
fingerprint = 'EState'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Caco2/features/Fingerprints/',fingerprint,'_train.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/Caco2/data/Train_caco2.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
EState_train = pd.concat([df_train[['ID','SMILES','Permeability']], descriptors], axis=1)
EState_train.drop(['Name'],axis=1,inplace=True)
EState_train.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Train/EState_train_Caco2.csv', index=False)
EState_train

,ID,SMILES,Permeability,EStateFP1,EStateFP2,EStateFP3,EStateFP4,EStateFP5,EStateFP6,EStateFP7,...,EStateFP70,EStateFP71,EStateFP72,EStateFP73,EStateFP74,EStateFP75,EStateFP76,EStateFP77,EStateFP78,EStateFP79
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1004,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1005,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1006,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [91]:
fingerprint = 'EState'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Caco2/features/Fingerprints/',fingerprint,'_test.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/Caco2/data/Test_caco2.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
EState_test = pd.concat([df_test[['ID','SMILES','Permeability']], descriptors], axis=1)
EState_test.drop(['Name'],axis=1,inplace=True)
EState_test.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Test/EState_test_Caco2.csv', index=False)
EState_test

,ID,SMILES,Permeability,EStateFP1,EStateFP2,EStateFP3,EStateFP4,EStateFP5,EStateFP6,EStateFP7,...,EStateFP70,EStateFP71,EStateFP72,EStateFP73,EStateFP74,EStateFP75,EStateFP76,EStateFP77,EStateFP78,EStateFP79
0,2064,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.19,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,8066,CC[C@H]1C(=O)N[C@@H](COCCC(C)C)C(=O)N(C)[C@@H]...,-6.21,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2068,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-7.24,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,2234,CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[...,-5.85,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,2230,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[...,-5.96,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,8448,CC(C)C[C@@H]1NC(=O)CN(C)C(=O)[C@H](Cc2ccccc2)N...,-5.88,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
248,2478,CC(C)C[C@@H]1NC(=O)[C@H](C)NCCCCCCNC(=O)[C@H](...,-4.50,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
249,2477,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-4.20,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
250,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [92]:
#Extended
fingerprint = 'Extended'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Caco2/features/Fingerprints/',fingerprint,'_train.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/Caco2/data/Train_caco2.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
Extended_train = pd.concat([df_train[['ID','SMILES','Permeability']], descriptors], axis=1)
Extended_train.drop(['Name'],axis=1,inplace=True)
Extended_train.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Train/Extended_train_Caco2.csv', index=False)
Extended_train

,ID,SMILES,Permeability,ExtFP1,ExtFP2,ExtFP3,ExtFP4,ExtFP5,ExtFP6,ExtFP7,...,ExtFP1015,ExtFP1016,ExtFP1017,ExtFP1018,ExtFP1019,ExtFP1020,ExtFP1021,ExtFP1022,ExtFP1023,ExtFP1024
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1004,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1005,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1006,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [93]:
fingerprint = 'Extended'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Caco2/features/Fingerprints/',fingerprint,'_test.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/Caco2/data/Test_caco2.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
Extended_test = pd.concat([df_test[['ID','SMILES','Permeability']], descriptors], axis=1)
Extended_test.drop(['Name'],axis=1,inplace=True)
Extended_test.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Test/Extended_test_Caco2.csv', index=False)
Extended_test

,ID,SMILES,Permeability,ExtFP1,ExtFP2,ExtFP3,ExtFP4,ExtFP5,ExtFP6,ExtFP7,...,ExtFP1015,ExtFP1016,ExtFP1017,ExtFP1018,ExtFP1019,ExtFP1020,ExtFP1021,ExtFP1022,ExtFP1023,ExtFP1024
0,2064,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.19,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,8066,CC[C@H]1C(=O)N[C@@H](COCCC(C)C)C(=O)N(C)[C@@H]...,-6.21,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2068,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-7.24,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
3,2234,CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[...,-5.85,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
4,2230,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[...,-5.96,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,8448,CC(C)C[C@@H]1NC(=O)CN(C)C(=O)[C@H](Cc2ccccc2)N...,-5.88,0,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
248,2478,CC(C)C[C@@H]1NC(=O)[C@H](C)NCCCCCCNC(=O)[C@H](...,-4.50,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
249,2477,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-4.20,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
250,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [94]:
#Fingerprinter
fingerprint = 'Fingerprinter'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Caco2/features/Fingerprints/',fingerprint,'_train.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/Caco2/data/Train_caco2.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
Fingerprinter_train = pd.concat([df_train[['ID','SMILES','Permeability']], descriptors], axis=1)
Fingerprinter_train.drop(['Name'],axis=1,inplace=True)
Fingerprinter_train.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Train/Fingerprinter_train_Caco2.csv', index=False)
Fingerprinter_train

,ID,SMILES,Permeability,FP1,FP2,FP3,FP4,FP5,FP6,FP7,...,FP1015,FP1016,FP1017,FP1018,FP1019,FP1020,FP1021,FP1022,FP1023,FP1024
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1004,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60,0,0,0,0,1,0,0,...,0,1,1,0,0,0,0,0,0,0
1005,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70,0,0,0,0,1,0,0,...,0,1,1,0,0,0,0,0,0,0
1006,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90,0,0,0,0,1,0,0,...,0,1,1,0,0,0,0,0,0,0


In [95]:
fingerprint = 'Fingerprinter'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Caco2/features/Fingerprints/',fingerprint,'_test.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/Caco2/data/Test_caco2.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
Fingerprinter_test = pd.concat([df_test[['ID','SMILES','Permeability']], descriptors], axis=1)
Fingerprinter_test.drop(['Name'],axis=1,inplace=True)
Fingerprinter_test.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Test/Fingerprinter_test_Caco2.csv', index=False)
Fingerprinter_test

,ID,SMILES,Permeability,FP1,FP2,FP3,FP4,FP5,FP6,FP7,...,FP1015,FP1016,FP1017,FP1018,FP1019,FP1020,FP1021,FP1022,FP1023,FP1024
0,2064,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.19,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8066,CC[C@H]1C(=O)N[C@@H](COCCC(C)C)C(=O)N(C)[C@@H]...,-6.21,0,0,0,0,1,0,0,...,1,1,0,0,0,0,0,0,0,0
2,2068,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-7.24,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,2234,CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[...,-5.85,0,0,0,0,1,0,0,...,0,1,1,0,0,0,0,0,0,0
4,2230,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[...,-5.96,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,8448,CC(C)C[C@@H]1NC(=O)CN(C)C(=O)[C@H](Cc2ccccc2)N...,-5.88,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
248,2478,CC(C)C[C@@H]1NC(=O)[C@H](C)NCCCCCCNC(=O)[C@H](...,-4.50,0,0,0,0,1,0,0,...,0,1,1,0,0,0,0,0,0,0
249,2477,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-4.20,0,0,0,0,1,0,0,...,0,1,1,0,0,0,0,0,0,0
250,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,0,0,0,0,1,0,0,...,0,1,1,0,0,0,0,0,0,0


In [96]:
#GraphOnly
fingerprint = 'Graphonly'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Caco2/features/Fingerprints/',fingerprint,'_train.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/Caco2/data/Train_caco2.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
Graphonly_train = pd.concat([df_train[['ID','SMILES','Permeability']], descriptors], axis=1)
Graphonly_train.drop(['Name'],axis=1,inplace=True)
Graphonly_train.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Train/Graphonly_train_Caco2.csv', index=False)
Graphonly_train

,ID,SMILES,Permeability,GraphFP1,GraphFP2,GraphFP3,GraphFP4,GraphFP5,GraphFP6,GraphFP7,...,GraphFP1015,GraphFP1016,GraphFP1017,GraphFP1018,GraphFP1019,GraphFP1020,GraphFP1021,GraphFP1022,GraphFP1023,GraphFP1024
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1004,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1005,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1006,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [97]:
fingerprint = 'Graphonly'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Caco2/features/Fingerprints/',fingerprint,'_test.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/Caco2/data/Test_caco2.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
Graphonly_test = pd.concat([df_test[['ID','SMILES','Permeability']], descriptors], axis=1)
Graphonly_test.drop(['Name'],axis=1,inplace=True)
Graphonly_test.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Test/Graphonly_test_Caco2.csv', index=False)
Graphonly_test

,ID,SMILES,Permeability,GraphFP1,GraphFP2,GraphFP3,GraphFP4,GraphFP5,GraphFP6,GraphFP7,...,GraphFP1015,GraphFP1016,GraphFP1017,GraphFP1018,GraphFP1019,GraphFP1020,GraphFP1021,GraphFP1022,GraphFP1023,GraphFP1024
0,2064,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.19,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8066,CC[C@H]1C(=O)N[C@@H](COCCC(C)C)C(=O)N(C)[C@@H]...,-6.21,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2068,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-7.24,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2234,CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[...,-5.85,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2230,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[...,-5.96,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,8448,CC(C)C[C@@H]1NC(=O)CN(C)C(=O)[C@H](Cc2ccccc2)N...,-5.88,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
248,2478,CC(C)C[C@@H]1NC(=O)[C@H](C)NCCCCCCNC(=O)[C@H](...,-4.50,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
249,2477,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-4.20,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
250,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [98]:
#KlekotaRothCount
fingerprint = 'KlekotaRothCount'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Caco2/features/Fingerprints/',fingerprint,'_train.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/Caco2/data/Train_caco2.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
KlekotaRothCount_train = pd.concat([df_train[['ID','SMILES','Permeability']], descriptors], axis=1)
KlekotaRothCount_train.drop(['Name'],axis=1,inplace=True)
KlekotaRothCount_train.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Train/KlekotaRothCount_train_Caco2.csv', index=False)
KlekotaRothCount_train

,ID,SMILES,Permeability,KRFPC1,KRFPC2,KRFPC3,KRFPC4,KRFPC5,KRFPC6,KRFPC7,...,KRFPC4851,KRFPC4852,KRFPC4853,KRFPC4854,KRFPC4855,KRFPC4856,KRFPC4857,KRFPC4858,KRFPC4859,KRFPC4860
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,18.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,18.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,16.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,18.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,14.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1004,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1005,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1006,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [99]:
fingerprint = 'KlekotaRothCount'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Caco2/features/Fingerprints/',fingerprint,'_test.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/Caco2/data/Test_caco2.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
KlekotaRothCount_test = pd.concat([df_test[['ID','SMILES','Permeability']], descriptors], axis=1)
KlekotaRothCount_test.drop(['Name'],axis=1,inplace=True)
KlekotaRothCount_test.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Test/KlekotaRothCount_test_Caco2.csv', index=False)
KlekotaRothCount_test

,ID,SMILES,Permeability,KRFPC1,KRFPC2,KRFPC3,KRFPC4,KRFPC5,KRFPC6,KRFPC7,...,KRFPC4851,KRFPC4852,KRFPC4853,KRFPC4854,KRFPC4855,KRFPC4856,KRFPC4857,KRFPC4858,KRFPC4859,KRFPC4860
0,2064,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.19,17.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8066,CC[C@H]1C(=O)N[C@@H](COCCC(C)C)C(=O)N(C)[C@@H]...,-6.21,16.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2068,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-7.24,16.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2234,CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[...,-5.85,16.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2230,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[...,-5.96,17.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,8448,CC(C)C[C@@H]1NC(=O)CN(C)C(=O)[C@H](Cc2ccccc2)N...,-5.88,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
248,2478,CC(C)C[C@@H]1NC(=O)[C@H](C)NCCCCCCNC(=O)[C@H](...,-4.50,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
249,2477,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-4.20,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
250,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [100]:
#KlekotaRoth
fingerprint = 'KlekotaRoth'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Caco2/features/Fingerprints/',fingerprint,'_train.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/Caco2/data/Train_caco2.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
KlekotaRoth_train = pd.concat([df_train[['ID','SMILES','Permeability']], descriptors], axis=1)
KlekotaRoth_train.drop(['Name'],axis=1,inplace=True)
KlekotaRoth_train.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Train/KlekotaRoth_train_Caco2.csv', index=False)
KlekotaRoth_train

,ID,SMILES,Permeability,KRFP1,KRFP2,KRFP3,KRFP4,KRFP5,KRFP6,KRFP7,...,KRFP4851,KRFP4852,KRFP4853,KRFP4854,KRFP4855,KRFP4856,KRFP4857,KRFP4858,KRFP4859,KRFP4860
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1004,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1005,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1006,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [101]:
fingerprint = 'KlekotaRoth'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Caco2/features/Fingerprints/',fingerprint,'_test.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/Caco2/data/Test_caco2.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
KlekotaRoth_test = pd.concat([df_test[['ID','SMILES','Permeability']], descriptors], axis=1)
KlekotaRoth_test.drop(['Name'],axis=1,inplace=True)
KlekotaRoth_test.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Test/KlekotaRoth_test_Caco2.csv', index=False)
KlekotaRoth_test

,ID,SMILES,Permeability,KRFP1,KRFP2,KRFP3,KRFP4,KRFP5,KRFP6,KRFP7,...,KRFP4851,KRFP4852,KRFP4853,KRFP4854,KRFP4855,KRFP4856,KRFP4857,KRFP4858,KRFP4859,KRFP4860
0,2064,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.19,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8066,CC[C@H]1C(=O)N[C@@H](COCCC(C)C)C(=O)N(C)[C@@H]...,-6.21,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2068,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-7.24,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2234,CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[...,-5.85,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2230,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[...,-5.96,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,8448,CC(C)C[C@@H]1NC(=O)CN(C)C(=O)[C@H](Cc2ccccc2)N...,-5.88,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
248,2478,CC(C)C[C@@H]1NC(=O)[C@H](C)NCCCCCCNC(=O)[C@H](...,-4.50,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
249,2477,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-4.20,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
250,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df_train_morgan = pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Train/morgan_fp_train_Caco2.csv')
df_train_morganCount = pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Train/count_morgan_fp_train_Caco2.csv')
df_train_AP2d = pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Train/AtomPairs2D_train_Caco2.csv')
df_train_AP2dCount = pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Train/AtomPairs2DCount_train_Caco2.csv')
df_train_EState = pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Train/EState_train_Caco2.csv')
df_train_Extended = pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Train/Extended_train_Caco2.csv')
df_train_fp= pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Train/Fingerprinter_train_Caco2.csv')
df_train_graph= pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Train/Graphonly_train_Caco2.csv')
df_train_kr= pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Train/KlekotaRoth_train_Caco2.csv')
df_train_krcount= pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Train/KlekotaRothCount_train_Caco2.csv')
df_train_maccs= pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Train/MACCS_train_Caco2.csv')
df_train_pubchem= pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Train/PubChem_train_Caco2.csv')
df_train_Substr= pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Train/Substructure_train_Caco2.csv')
df_train_Substrcount= pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Train/SubstructureCount_train_Caco2.csv')

df = df_train_morgan.merge(df_train_morganCount, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_train_AP2d, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_train_AP2dCount, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_train_EState, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_train_Extended, on=['ID', 'SMILES', 'Permeability'], how='inner')
df = df.merge(df_train_fp, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_train_graph, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_train_kr, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_train_krcount, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_train_maccs, on=['ID', 'SMILES', 'Permeability'], how='inner')
df = df.merge(df_train_pubchem, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_train_Substr, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_train_Substrcount, on=['ID', 'SMILES', 'Permeability'], how='inner')
df

,ID,SMILES,Permeability,Morgan_fp_0,Morgan_fp_1,Morgan_fp_2,Morgan_fp_3,Morgan_fp_4,Morgan_fp_5,Morgan_fp_6,...,SubFPC298,SubFPC299,SubFPC300,SubFPC301,SubFPC302,SubFPC303,SubFPC304,SubFPC305,SubFPC306,SubFPC307
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,0,1,1,0,0,1,0,...,0.0,0.0,77.0,77.0,28.0,0.0,0.0,0.0,0.0,70.0
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,0,1,1,0,0,1,0,...,0.0,0.0,77.0,77.0,28.0,0.0,0.0,0.0,0.0,70.0
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,0,1,1,0,0,1,0,...,0.0,0.0,76.0,76.0,24.0,0.0,0.0,0.0,0.0,55.0
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,0,1,1,0,0,1,0,...,0.0,0.0,75.0,75.0,25.0,0.0,0.0,0.0,0.0,64.0
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,0,1,1,0,0,1,0,...,0.0,0.0,76.0,76.0,26.0,0.0,0.0,0.0,0.0,53.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,0,1,0,0,0,1,0,...,0.0,0.0,21.0,21.0,4.0,0.0,0.0,0.0,0.0,17.0
1004,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60,0,1,1,0,0,0,0,...,0.0,0.0,22.0,22.0,4.0,0.0,0.0,0.0,0.0,17.0
1005,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70,0,1,0,0,0,0,0,...,0.0,0.0,21.0,21.0,4.0,0.0,0.0,0.0,0.0,17.0
1006,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90,0,1,0,0,0,0,0,...,0.0,0.0,20.0,20.0,4.0,0.0,0.0,0.0,0.0,17.0


In [4]:
df.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Train/All_fingerprints_train_Caco2.csv',index=False)

In [5]:
df_test_morgan = pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Test/morgan_fp_test_Caco2.csv')
df_test_morganCount = pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Test/count_morgan_fp_test_Caco2.csv')
df_test_AP2d = pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Test/AtomPairs2D_test_Caco2.csv')
df_test_AP2dCount = pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Test/AtomPairs2DCount_test_Caco2.csv')
df_test_EState = pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Test/EState_test_Caco2.csv')
df_test_Extended = pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Test/Extended_test_Caco2.csv')
df_test_fp= pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Test/Fingerprinter_test_Caco2.csv')
df_test_graph= pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Test/Graphonly_test_Caco2.csv')
df_test_kr= pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Test/KlekotaRoth_test_Caco2.csv')
df_test_krcount= pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Test/KlekotaRothCount_test_Caco2.csv')
df_test_maccs= pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Test/MACCS_test_Caco2.csv')
df_test_pubchem= pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Test/PubChem_test_Caco2.csv')
df_test_Substr= pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Test/Substructure_test_Caco2.csv')
df_test_Substrcount= pd.read_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Test/SubstructureCount_test_Caco2.csv')

df = df_test_morgan.merge(df_test_morganCount, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_test_AP2d, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_test_AP2dCount, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_test_EState, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_test_Extended, on=['ID', 'SMILES', 'Permeability'], how='inner')
df = df.merge(df_test_fp, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_test_graph, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_test_kr, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_test_krcount, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_test_maccs, on=['ID', 'SMILES', 'Permeability'], how='inner')
df = df.merge(df_test_pubchem, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_test_Substr, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_test_Substrcount, on=['ID', 'SMILES', 'Permeability'], how='inner')
df

,ID,SMILES,Permeability,Morgan_fp_0,Morgan_fp_1,Morgan_fp_2,Morgan_fp_3,Morgan_fp_4,Morgan_fp_5,Morgan_fp_6,...,SubFPC298,SubFPC299,SubFPC300,SubFPC301,SubFPC302,SubFPC303,SubFPC304,SubFPC305,SubFPC306,SubFPC307
0,2064,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.19,0,1,1,0,0,1,0,...,0.0,0.0,77.0,77.0,28.0,0.0,0.0,0.0,0.0,69.0
1,8066,CC[C@H]1C(=O)N[C@@H](COCCC(C)C)C(=O)N(C)[C@@H]...,-6.21,0,1,1,0,0,1,0,...,0.0,0.0,89.0,89.0,32.0,0.0,0.0,0.0,0.0,48.0
2,2068,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-7.24,0,1,1,0,0,1,0,...,0.0,0.0,73.0,73.0,22.0,0.0,0.0,0.0,0.0,65.0
3,2234,CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[...,-5.85,0,1,1,0,0,1,0,...,0.0,0.0,77.0,77.0,17.0,0.0,0.0,0.0,0.0,55.0
4,2230,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[...,-5.96,0,1,1,0,0,1,0,...,0.0,0.0,80.0,80.0,19.0,0.0,0.0,0.0,0.0,56.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,8448,CC(C)C[C@@H]1NC(=O)CN(C)C(=O)[C@H](Cc2ccccc2)N...,-5.88,0,1,0,0,0,1,0,...,0.0,0.0,19.0,19.0,4.0,0.0,0.0,0.0,0.0,20.0
248,2478,CC(C)C[C@@H]1NC(=O)[C@H](C)NCCCCCCNC(=O)[C@H](...,-4.50,0,1,1,0,0,0,0,...,0.0,0.0,23.0,23.0,4.0,0.0,0.0,0.0,0.0,17.0
249,2477,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-4.20,0,1,1,0,0,0,0,...,0.0,0.0,23.0,23.0,4.0,0.0,0.0,0.0,0.0,17.0
250,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,0,1,1,0,0,0,0,...,0.0,0.0,23.0,23.0,4.0,0.0,0.0,0.0,0.0,17.0


In [6]:
df.to_csv('/home/users/akshay/PCPpred/Caco2/features/Fingerprints/Test/All_fingerprints_test_Caco2.csv',index=False)

In [106]:
#Descriptors
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors, Descriptors3D
from rdkit.ML.Descriptors import MoleculeDescriptors
from tqdm import tqdm

In [107]:
calc = MoleculeDescriptors.MolecularDescriptorCalculator([desc[0] for desc in Chem.Descriptors._descList])

def calculate_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None 
    return calc.CalcDescriptors(mol)

In [108]:
#2D RDKit train dataset descriptors
df_train = pd.read_csv("/home/users/akshay/PCPpred/Caco2/data/Train_Caco2.csv")
descriptor_data = []
for smiles in df_train['SMILES']:
    descriptors = calculate_descriptors(smiles)
    if descriptors is not None:
        descriptor_data.append(descriptors)
    else:
        descriptor_data.append([np.nan] * len(calc.GetDescriptorNames()))

descriptor_df = pd.DataFrame(descriptor_data, columns=calc.GetDescriptorNames())
train_2d_rdkit = pd.concat([df_train[['ID','SMILES','Permeability']], descriptor_df], axis=1)
train_2d_rdkit.to_csv('/home/users/akshay/PCPpred/Caco2/features/Descriptors/Train_2d_RDKit_des_Caco2.csv',index=False)
print("Shape: ",train_2d_rdkit.shape, '\n')
train_2d_rdkit

Shape:  (1008, 220) 



,ID,SMILES,Permeability,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,16.271931,16.271931,0.025756,-1.930898,0.058792,23.408333,1664.156,...,0,0,0,0,0,0,0,0,0,0
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,16.083777,16.083777,0.027668,-1.934865,0.053531,23.411765,1650.129,...,0,0,0,0,0,0,0,0,0,0
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,15.668220,15.668220,0.003714,-4.150777,0.085546,24.018349,1587.863,...,0,1,0,0,0,0,0,0,1,0
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,16.048909,16.048909,0.025704,-1.891709,0.101375,24.159292,1574.031,...,0,0,0,0,0,0,0,0,0,0
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,15.511700,15.511700,0.033138,-4.150544,0.085338,23.366972,1567.445,...,0,1,0,0,0,0,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,12.898424,12.898424,0.143657,-0.742398,0.662387,22.258065,430.549,...,0,0,0,0,0,0,0,0,0,0
1004,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60,12.903074,12.903074,0.161549,-0.731330,0.598056,24.933333,416.566,...,0,0,0,0,0,0,0,0,0,0
1005,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70,12.835172,12.835172,0.168936,-0.728726,0.606745,24.965517,402.539,...,0,0,0,0,0,0,0,0,0,0
1006,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90,12.763992,12.763992,0.179945,-0.726122,0.611430,25.000000,388.512,...,0,0,0,0,0,0,0,0,0,0


In [109]:
#2D RDKit test dataset descriptors
df_test = pd.read_csv("/home/users/akshay/PCPpred/Caco2/data/Test_Caco2.csv")
descriptor_data = []
for smiles in df_test['SMILES']:
    descriptors = calculate_descriptors(smiles)
    if descriptors is not None:
        descriptor_data.append(descriptors)
    else:
        descriptor_data.append([np.nan] * len(calc.GetDescriptorNames()))

descriptor_df = pd.DataFrame(descriptor_data, columns=calc.GetDescriptorNames())
test_2d_rdkit = pd.concat([df_test[['ID','SMILES','Permeability']], descriptor_df], axis=1)
test_2d_rdkit.to_csv('/home/users/akshay/PCPpred/Caco2/features/Descriptors/Test_2d_RDKit_des_Caco2.csv',index=False)
print("Shape: ",test_2d_rdkit.shape, '\n')
test_2d_rdkit

Shape:  (252, 220) 



,ID,SMILES,Permeability,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,2064,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.19,15.976055,15.976055,0.030813,-1.935404,0.058779,22.883333,1664.156,...,0,0,0,0,0,0,0,0,0,0
1,8066,CC[C@H]1C(=O)N[C@@H](COCCC(C)C)C(=O)N(C)[C@@H]...,-6.21,15.516152,15.516152,0.000091,-1.679143,0.049536,24.360360,1572.049,...,0,0,0,0,0,0,0,0,0,0
2,2068,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-7.24,15.993973,15.993973,0.020362,-1.750211,0.057933,24.294643,1552.024,...,0,0,0,0,0,0,0,0,0,0
3,2234,CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[...,-5.85,15.804049,15.804049,0.002346,-2.970606,0.165351,26.981481,1537.344,...,0,0,0,0,0,0,0,0,0,0
4,2230,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[...,-5.96,15.545371,15.545371,0.013183,-2.948188,0.126607,27.422018,1530.953,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,8448,CC(C)C[C@@H]1NC(=O)CN(C)C(=O)[C@H](Cc2ccccc2)N...,-5.88,13.192828,13.192828,0.126957,-0.677034,0.670121,21.000000,440.544,...,0,0,0,0,0,0,0,0,0,0
248,2478,CC(C)C[C@@H]1NC(=O)[C@H](C)NCCCCCCNC(=O)[C@H](...,-4.50,13.114656,13.114656,0.186152,-0.689890,0.586385,24.903226,430.593,...,0,0,0,0,0,0,0,0,0,0
249,2477,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-4.20,13.174810,13.174810,0.158364,-0.755076,0.586385,24.903226,430.593,...,0,0,0,0,0,0,0,0,0,0
250,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,12.958074,12.958074,0.156212,-0.733388,0.586385,24.903226,430.593,...,0,0,0,0,0,0,0,0,0,0


In [110]:
from mordred import Calculator, descriptors
calc = Calculator(descriptors, ignore_3D=True)

def calculate_mordred_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None  # Return None if the SMILES is invalid
    return calc(mol)

In [111]:
print('start')
#2D Mordred train descriptors
df_train = pd.read_csv("/home/users/akshay/PCPpred/Caco2/data/Train_Caco2.csv")
descriptor_data = []
for smiles in df_train['SMILES']:
    descriptors = calculate_mordred_descriptors(smiles)
    if descriptors is not None:
        descriptor_data.append(descriptors)
    else:
        descriptor_data.append([np.nan] * len(calc.descriptors))

descriptor_df = pd.DataFrame(descriptor_data, columns=[desc.__class__.__name__ for desc in calc.descriptors])
train_mordred_2d = pd.concat([df_train[['ID','SMILES','Permeability']], descriptor_df], axis=1)
train_mordred_2d.to_csv('/home/users/akshay/PCPpred/Caco2/features/Descriptors/Train_2d_Mordred_desc_caco2.csv', index=False)
print('Shape: ',train_mordred_2d.shape)
train_mordred_2d

start


/home/users/akshay/anaconda3/envs/cycpep/lib/python3.12/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Shape:  (1008, 1616)


,ID,SMILES,Permeability,ABCIndex,ABCGGIndex,AcidicGroupCount,BasicGroupCount,AdjacencyMatrix,AdjacencyMatrix,AdjacencyMatrix,...,WalkCount,WalkCount,Weight,Weight,WienerIndex,WienerIndex,ZagrebIndex,ZagrebIndex,ZagrebIndex,ZagrebIndex
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,150.002370,2.434280,4.865870,...,11.590451,162.312312,1663.015310,6.547304,91297,211,602.0,703.0,50.611111,26.777778
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,148.508426,2.439658,4.878720,...,11.569485,161.240399,1648.999660,6.569720,90388,207,596.0,694.0,49.750000,26.555556
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,130.882818,2.423927,4.847855,...,11.437501,150.677817,1585.815201,6.894849,69754,181,540.0,618.0,48.506944,24.125000
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,140.033288,2.434167,4.866917,...,11.543988,155.034166,1572.968359,6.526840,79884,202,566.0,663.0,49.750000,25.222222
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,132.454775,2.413793,4.827585,...,11.409840,150.603018,1565.869824,6.720471,69806,179,536.0,612.0,47.284722,24.347222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,37.855097,2.317338,4.629714,...,9.940542,65.841858,430.258006,6.619354,2644,45,148.0,164.0,11.750000,7.000000
1004,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,1,37.048032,2.303371,4.602630,...,9.872152,64.585516,416.278741,6.307254,2503,43,142.0,157.0,10.888889,6.833333
1005,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,1,35.896500,2.304816,4.609632,...,9.859065,63.474076,402.263091,6.385128,2286,42,138.0,153.0,10.638889,6.583333
1006,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,1,34.487329,2.307278,4.602399,...,9.845805,62.359842,388.247441,6.470791,2069,41,134.0,149.0,10.388889,6.333333


In [112]:
#2D Mordred test descriptors
df_test = pd.read_csv("/home/users/akshay/PCPpred/Caco2/data/Test_Caco2.csv")
descriptor_data = []
for smiles in df_test['SMILES']:
    descriptors = calculate_mordred_descriptors(smiles)
    if descriptors is not None:
        descriptor_data.append(descriptors)
    else:
        descriptor_data.append([np.nan] * len(calc.descriptors))

descriptor_df = pd.DataFrame(descriptor_data, columns=[desc.__class__.__name__ for desc in calc.descriptors])
test_mordred_2d = pd.concat([df_test[['ID','SMILES','Permeability']], descriptor_df], axis=1)
test_mordred_2d.to_csv('/home/users/akshay/PCPpred/Caco2/features/Descriptors/Test_2d_Mordred_desc_Caco2.csv', index=False)
print('Shape: ',test_mordred_2d.shape)
test_mordred_2d

/home/users/akshay/anaconda3/envs/cycpep/lib/python3.12/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Shape:  (252, 1616)


,ID,SMILES,Permeability,ABCIndex,ABCGGIndex,AcidicGroupCount,BasicGroupCount,AdjacencyMatrix,AdjacencyMatrix,AdjacencyMatrix,...,WalkCount,WalkCount,Weight,Weight,WienerIndex,WienerIndex,ZagrebIndex,ZagrebIndex,ZagrebIndex,ZagrebIndex
0,2064,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.19,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,150.071798,2.429231,4.858462,...,11.569599,162.262107,1663.015310,6.547304,94190,208,600.0,698.0,50.000000,26.805556
1,8066,CC[C@H]1C(=O)N[C@@H](COCCC(C)C)C(=O)N(C)[C@@H]...,-6.21,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,133.544218,2.399797,4.799594,...,11.363578,152.529489,1571.020120,6.309318,74798,175,538.0,606.0,48.763889,24.777778
2,2068,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-7.24,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,138.687365,2.431377,4.862754,...,11.482930,153.892806,1550.951647,6.516604,73336,187,562.0,647.0,45.645833,24.736111
3,2234,CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[...,-5.85,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,1,131.842973,2.484317,4.905149,...,11.534364,163.933604,1535.873430,6.736287,67356,187,554.0,645.0,46.569444,23.486111
4,2230,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[...,-5.96,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,1,133.583282,2.474028,4.901032,...,11.515712,164.907694,1529.943702,6.538221,69591,185,556.0,644.0,46.208333,23.763889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,8448,CC(C)C[C@@H]1NC(=O)CN(C)C(=O)[C@H](Cc2ccccc2)N...,-5.88,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,1,40.208444,2.339628,4.650167,...,10.071076,80.884603,440.242356,6.878787,2857,47,160.0,181.0,11.111111,7.083333
248,2478,CC(C)C[C@@H]1NC(=O)[C@H](C)NCCCCCCNC(=O)[C@H](...,-4.50,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,1,38.475166,2.317701,4.635401,...,9.885578,65.694814,430.294391,6.236151,2660,44,146.0,161.0,11.138889,7.083333
249,2477,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-4.20,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,1,38.474998,2.320224,4.640447,...,9.885578,65.694814,430.294391,6.236151,2648,44,146.0,161.0,11.138889,7.083333
250,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,1,38.436032,2.302525,4.605049,...,9.885069,65.694305,430.294391,6.236151,2750,44,146.0,161.0,11.138889,7.083333


In [113]:
#RDKit 3d descriptors
def generate_3d_descriptors(smiles):
    # Convert SMILES to a molecule object
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        print(f"Invalid SMILES: {smiles}")
        return None
    
    # Add hydrogens
    mol = Chem.AddHs(mol)
    
    # Generate 3D coordinates for the molecule
    AllChem.EmbedMolecule(mol)
    
    try:
        descriptors = Descriptors3D.CalcMolDescriptors3D(mol)
        return descriptors
    except Exception as e:
        print(f"Error calculating descriptors for SMILES '{smiles}': {e}")
        return None

In [114]:
df_train = pd.read_csv("/home/users/akshay/PCPpred/Caco2/data/Train_Caco2.csv")
descriptor_data = []
for smiles in tqdm(df_train['SMILES'],desc='3d_descriptors', unit='smiles'):
    descriptors =generate_3d_descriptors(smiles) 
    if descriptors is not None:
        descriptor_data.append(descriptors)
    else:
        descriptor_data.append({'PMI1': np.nan,
  'PMI2': np.nan,
  'PMI3': np.nan,
  'NPR1': np.nan,
  'NPR2': np.nan,
  'RadiusOfGyration': np.nan,
  'InertialShapeFactor': np.nan,
  'Eccentricity': np.nan,
  'Asphericity': np.nan,
  'SpherocityIndex': np.nan,
  'PBF': np.nan})

3d_descriptors: 100%|██████████| 1008/1008 [45:39<00:00,  2.72s/smiles] 


In [115]:
descriptor_df = pd.DataFrame(descriptor_data)
descriptor_df

,PMI1,PMI2,PMI3,NPR1,NPR2,RadiusOfGyration,InertialShapeFactor,Eccentricity,Asphericity,SpherocityIndex,PBF
0,44715.312675,91107.809558,123619.238351,0.361718,0.737003,8.828937,0.000016,0.932288,0.280346,0.176547,1.794050
1,39878.864245,83243.633963,109538.317627,0.364063,0.759950,8.396295,0.000019,0.931374,0.274310,0.191113,1.736770
2,38542.444878,76292.960568,100389.431793,0.383929,0.759970,8.232366,0.000020,0.923362,0.251752,0.285578,2.034590
3,34566.515473,82504.806655,101941.820182,0.339081,0.809332,8.340910,0.000023,0.940757,0.300846,0.242195,2.018193
4,43675.463614,58178.760645,90645.390361,0.481828,0.641828,7.836169,0.000015,0.876266,0.187316,0.223256,1.773755
...,...,...,...,...,...,...,...,...,...,...,...
1003,3108.457288,3620.937641,5561.408951,0.558933,0.651083,3.778017,0.000209,0.829213,0.132991,0.286191,1.038669
1004,1705.198171,6092.708948,7027.507516,0.242646,0.866980,4.218388,0.000508,0.970115,0.440879,0.166539,0.926130
1005,1587.669882,5846.537779,6808.341490,0.233195,0.858732,4.206054,0.000541,0.972430,0.456676,0.175519,0.951673
1006,1686.370397,5331.220745,6490.249963,0.259831,0.821420,4.169421,0.000487,0.965654,0.413298,0.166015,0.886085


In [116]:
num_columns = descriptor_df.shape[1]
descriptor_df.columns = [f'3d_rdkit_{i+1}' for i in range(num_columns)]
train_3d_rdkit = pd.concat([df_train[['ID','SMILES','Permeability']], descriptor_df], axis=1)
print('Shape before:' , train_3d_rdkit.shape)
train_3d_rdkit.to_csv('/home/users/akshay/PCPpred/Caco2/features/Descriptors/Train_3d_RDKit_desc_Caco2.csv',index=False)
print('Shape after: ',train_3d_rdkit.shape)
train_3d_rdkit

Shape before: (1008, 14)
Shape after:  (1008, 14)


,ID,SMILES,Permeability,3d_rdkit_1,3d_rdkit_2,3d_rdkit_3,3d_rdkit_4,3d_rdkit_5,3d_rdkit_6,3d_rdkit_7,3d_rdkit_8,3d_rdkit_9,3d_rdkit_10,3d_rdkit_11
0,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,44715.312675,91107.809558,123619.238351,0.361718,0.737003,8.828937,0.000016,0.932288,0.280346,0.176547,1.794050
1,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,39878.864245,83243.633963,109538.317627,0.364063,0.759950,8.396295,0.000019,0.931374,0.274310,0.191113,1.736770
2,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,38542.444878,76292.960568,100389.431793,0.383929,0.759970,8.232366,0.000020,0.923362,0.251752,0.285578,2.034590
3,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,34566.515473,82504.806655,101941.820182,0.339081,0.809332,8.340910,0.000023,0.940757,0.300846,0.242195,2.018193
4,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,43675.463614,58178.760645,90645.390361,0.481828,0.641828,7.836169,0.000015,0.876266,0.187316,0.223256,1.773755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,3108.457288,3620.937641,5561.408951,0.558933,0.651083,3.778017,0.000209,0.829213,0.132991,0.286191,1.038669
1004,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60,1705.198171,6092.708948,7027.507516,0.242646,0.866980,4.218388,0.000508,0.970115,0.440879,0.166539,0.926130
1005,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70,1587.669882,5846.537779,6808.341490,0.233195,0.858732,4.206054,0.000541,0.972430,0.456676,0.175519,0.951673
1006,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90,1686.370397,5331.220745,6490.249963,0.259831,0.821420,4.169421,0.000487,0.965654,0.413298,0.166015,0.886085


In [117]:
df_test = pd.read_csv("/home/users/akshay/PCPpred/Caco2/data/Test_Caco2.csv")
descriptor_data = []
for smiles in tqdm(df_test['SMILES'],desc='3d_descriptors', unit='smiles'):
    descriptors = generate_3d_descriptors(smiles)
    if descriptors is not None:
        descriptor_data.append(descriptors)
    else:
        descriptor_data.append({'PMI1': np.nan,
  'PMI2': np.nan,
  'PMI3': np.nan,
  'NPR1': np.nan,
  'NPR2': np.nan,
  'RadiusOfGyration': np.nan,
  'InertialShapeFactor': np.nan,
  'Eccentricity': np.nan,
  'Asphericity': np.nan,
  'SpherocityIndex': np.nan,
  'PBF': np.nan})

descriptor_data

3d_descriptors: 100%|██████████| 252/252 [11:58<00:00,  2.85s/smiles]


[{'PMI1': 39993.25422337978,
  'PMI2': 103505.49077897394,
  'PMI3': 135075.8476930671,
  'NPR1': 0.29607997955531223,
  'NPR2': 0.7662768181486412,
  'RadiusOfGyration': 9.148686308456169,
  'InertialShapeFactor': 1.9160151706301536e-05,
  'Eccentricity': 0.9551631513550581,
  'Asphericity': 0.3626417917380852,
  'SpherocityIndex': 0.1192870954377636,
  'PBF': 1.5226975070625883},
 {'PMI1': 34076.71240186463,
  'PMI2': 75970.79219715757,
  'PMI3': 94707.19014135368,
  'NPR1': 0.3598112492937863,
  'NPR2': 0.8021649896250601,
  'RadiusOfGyration': 8.069913822854371,
  'InertialShapeFactor': 2.353997592740686e-05,
  'Eccentricity': 0.9330251148182693,
  'Asphericity': 0.2758398276435739,
  'SpherocityIndex': 0.27032777737045655,
  'PBF': 2.0789541859399927},
 {'PMI1': 39130.18449944606,
  'PMI2': 55100.82542366506,
  'PMI3': 80924.80560391583,
  'NPR1': 0.48353757797043895,
  'NPR2': 0.6808891910516843,
  'RadiusOfGyration': 7.5118700521049995,
  'InertialShapeFactor': 1.740061284559809

In [118]:
descriptor_df = pd.DataFrame(descriptor_data)

In [119]:
num_columns = descriptor_df.shape[1]
print('num_columns',num_columns)
descriptor_df.columns = [f'3d_rdkit_{i+1}' for i in range(num_columns)]
print('descriptor_df.columns',descriptor_df.columns)
test_3d_rdkit = pd.concat([df_test[['ID','SMILES','Permeability']], descriptor_df], axis=1)
print('Shape before:' , test_3d_rdkit.shape)
test_3d_rdkit.to_csv('/home/users/akshay/PCPpred/Caco2/features/Descriptors/Test_3d_RDKit_desc_Caco2.csv',index=False)
print('Shape after: ',test_3d_rdkit.shape)
test_3d_rdkit

num_columns 11
descriptor_df.columns Index(['3d_rdkit_1', '3d_rdkit_2', '3d_rdkit_3', '3d_rdkit_4', '3d_rdkit_5',
       '3d_rdkit_6', '3d_rdkit_7', '3d_rdkit_8', '3d_rdkit_9', '3d_rdkit_10',
       '3d_rdkit_11'],
      dtype='object')
Shape before: (252, 14)
Shape after:  (252, 14)


,ID,SMILES,Permeability,3d_rdkit_1,3d_rdkit_2,3d_rdkit_3,3d_rdkit_4,3d_rdkit_5,3d_rdkit_6,3d_rdkit_7,3d_rdkit_8,3d_rdkit_9,3d_rdkit_10,3d_rdkit_11
0,2064,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.19,39993.254223,103505.490779,135075.847693,0.296080,0.766277,9.148686,0.000019,0.955163,0.362642,0.119287,1.522698
1,8066,CC[C@H]1C(=O)N[C@@H](COCCC(C)C)C(=O)N(C)[C@@H]...,-6.21,34076.712402,75970.792197,94707.190141,0.359811,0.802165,8.069914,0.000024,0.933025,0.275840,0.270328,2.078954
2,2068,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-7.24,39130.184499,55100.825424,80924.805604,0.483538,0.680889,7.511870,0.000017,0.875324,0.173974,0.270244,1.932516
3,2234,CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[...,-5.85,44470.276090,60415.318636,87212.902057,0.509905,0.692734,7.904264,0.000016,0.860231,0.151715,0.300718,2.035308
4,2230,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[...,-5.96,40729.464320,63771.554832,88096.437834,0.462328,0.723883,7.931026,0.000018,0.886709,0.181500,0.295682,2.090568
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,8448,CC(C)C[C@@H]1NC(=O)CN(C)C(=O)[C@H](Cc2ccccc2)N...,-5.88,2624.075321,5553.550321,7616.331460,0.344533,0.729163,4.233854,0.000278,0.938774,0.302743,0.160928,0.852819
248,2478,CC(C)C[C@@H]1NC(=O)[C@H](C)NCCCCCCNC(=O)[C@H](...,-4.50,2849.828808,4851.229336,6659.776052,0.427917,0.728437,4.083583,0.000256,0.903818,0.211335,0.256039,1.090218
249,2477,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-4.20,3315.000108,4300.114122,6782.316344,0.488771,0.634019,4.088783,0.000191,0.872412,0.184808,0.240248,1.034360
250,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,3105.006206,4093.853252,5671.614623,0.547464,0.721814,3.865884,0.000232,0.836829,0.121397,0.313281,1.176090


In [141]:
import os

#Train MOL structure generator
input_file = '/home/users/akshay/PCPpred/Caco2/data/Train_caco2.smi'
output_dir = '/home/users/akshay/PCPpred/Caco2/data/Train_mol_Caco2'

os.makedirs(output_dir, exist_ok=True)

# Read the .smi file
with open(input_file, 'r') as f:
    lines = f.readlines()

for line in tqdm(lines, desc="Processing molecules"):
    parts = line.strip().split()
    if len(parts) != 2:
        continue  
    smiles, ID = parts

    # Generate 3D conformation
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        print(f"Invalid SMILES: {smiles} for id: {ID}")
        continue

    # Add hydrogens and generate 3D coordinates
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol)
    

    # Save the 3D conformation in MDL format
    output_file = os.path.join(output_dir, f"{ID}.mdl")
    with open(output_file, 'w') as out:
        out.write(Chem.MolToMolBlock(mol))

print("3D conformations generated and saved.")

Processing molecules: 100%|██████████| 1008/1008 [50:07<00:00,  2.98s/it] 

3D conformations generated and saved.


In [142]:
directory_path = '/home/users/akshay/PCPpred/Caco2/data/Train_mol_Caco2'
files = os.listdir(directory_path)
file_count = sum(os.path.isfile(os.path.join(directory_path, f)) for f in files)

print(f"Number of files in the directory: {file_count}")

Number of files in the directory: 1008


In [143]:
#Test MOL structure generator
input_file = '/home/users/akshay/PCPpred/Caco2/data/Test_caco2.smi'
output_dir = '/home/users/akshay/PCPpred/Caco2/data/Test_mol_Caco2'

os.makedirs(output_dir, exist_ok=True)

# Read the .smi file
with open(input_file, 'r') as f:
    lines = f.readlines()

for line in tqdm(lines, desc="Processing molecules"):
    parts = line.strip().split()
    if len(parts) != 2:
        continue  
    smiles, ID = parts

    # Generate 3D conformation
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        print(f"Invalid SMILES: {smiles} for id: {ID}")
        continue

    # Add hydrogens and generate 3D coordinates
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol)
    

    # Save the 3D conformation in MDL format
    output_file = os.path.join(output_dir, f"{ID}.mdl")
    with open(output_file, 'w') as out:
        out.write(Chem.MolToMolBlock(mol))

print("3D conformations generated and saved.")

Processing molecules: 100%|██████████| 252/252 [13:08<00:00,  3.13s/it]

3D conformations generated and saved.


In [144]:
directory_path = '/home/users/akshay/PCPpred/Caco2/data/Test_mol_Caco2'
files = os.listdir(directory_path)
file_count = sum(os.path.isfile(os.path.join(directory_path, f)) for f in files)

print(f"Number of files in the directory: {file_count}")

Number of files in the directory: 252
